<font size="6">Линейные модели</font>

Сегодня мы познакомимся с линейными моделями. Эти модели являются частью нейронных сетей, которым посвящена существенная часть нашего курса.


В основе линейных моделей лежит линейная функция
$$\large f(\vec x) = (\vec w, \vec x) +b, $$
где $(\vec w, \vec x)$ — скалярное произведение:

$$\large (\vec w, \vec x) = \sum_{i=1}^n{w_ix_i} = w_1x_1+w_2x_2+...+w_ix_i+ ... + w_nx_n$$

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/scalar_product_ways_to_use.png" width="800">

# Линейная регрессия

**Регрессия** — это одна из трех базовых задач машинного обучения (классификация, регрессия, кластеризация).

В задаче **регрессии** мы используем входные **признаки**, чтобы предсказать **целевые значения**. Например, чтобы предсказать температуру воздуха над населённым пунктом на следующий день, если известны различные характеристики на текущий момент времени (влажность воздуха, скорость и направление ветра, плотность застройки или наличие промышленных объектов, и т.п.)


##  Модель и ее параметры

Предположим, у нас есть набор точек $\{(x_i, y_i)\}$.

Цель линейной регрессии — **поиск линии, которая наилучшим образом соответствует заданным точкам**. Напомним, что уравнение прямой:

$$\large f(x) = w⋅x + b,$$

где $w$ — характеризует наклон линии (в будущем мы будем называть значения $w$ весом, weight) а $b$ — её сдвиг по оси $y$ (bias). Таким образом, решение линейной регрессии определяет значения для $w$ и $b$ так, что $f(x)$ приближается как можно ближе к $y(x)$. Здесь $w$ и $b$ — **параметры модели**.

Отобразим на графике случайные точки, расположенные в окрестности $y(x) = 3⋅x + 2$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(42)
x = np.random.rand(100, 1)
y = 2 + 3 * x + (np.random.rand(100, 1) - 0.5)

plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Предположим, что нам неизвестны параметры наклона и сдвига $w$ и $b$. Для их определения мы бы могли рассмотреть все возможные прямые вида $f(x) = w⋅x + b$ и выбрать среди семейства прямых такую, которая лучше всего приближает имеющиеся данные:

In [ ]:
plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
for w in np.arange(-5.0, 7.0, 1):
    for b in [-1, 0, 1, 2, 3]:
        y_predicted = b + w * x
        plt.plot(x, y_predicted, color="r", alpha=0.3)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

**Модель** $f(x) = w⋅x + b$ задаёт параметрическое семейство функций, а **выбор "правильного" представителя** из **параметрического семейства** и называется **обучением** модели:

In [ ]:
plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
for w in np.arange(-5.0, 7.0, 1):
    for b in [-1, 0, 1, 2, 3]:
        y_predicted = b + w * x
        plt.plot(x, y_predicted, color="r", alpha=0.3)
plt.plot(x, 2 + 3 * x, color="g")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Функция потерь

Как выбрать параметры?

**Функция потерь** позволяет вычислить меру количества ошибок. Для задачи **регрессии** такой мерой может быть **расстояние** между предсказанным значением $f(x)$ и его фактическим значением. Распространенной функцией потерь является **средняя квадратичная ошибка** (MSE).

Мы определяем ошибку модели на одном объекте как квадрат расстояния $l_i$ между предсказанием $f(x_i)$ и истинным значением $y_i$, а общая функция потерь будет задана выражением:

$$l_i =|f(x_i)-y_i| $$

$$ \text{Loss} = \frac{1}{N}\sum l_i^2 = \frac{1}{N} \sum (f(x_i)-y_i)^2$$

Для прямой с параметрами $w=4$, $b = 2$ и $w=3$, $b = 2$ (верные значения):

In [ ]:
def plot_delta_line(ax, x, y, w, b, color="r"):
    y_predicted = w * x + b
    # line
    ax.plot(x, y_predicted, color=color, alpha=0.5, label=f"f(x)={w}x+{b}")
    # delta
    for x_i, y_i, f_x in zip(x, y, y_predicted):
        ax.vlines(x=x_i, ymin=min(f_x, y_i), ymax=max(f_x, y_i), ls="--", alpha=0.3)
    # MSE
    loss = np.sum((y - (w * x + b)) ** 2) / (len(x))
    ax.set_title(f"MSE = {loss:.3f}")
    ax.legend()


fig, axs = plt.subplots(1, 2, figsize=(11, 4))

# plot x_i y_i (dots)
for ax in axs:
    ax.scatter(x, y, s=10)
    ax.set_xlim([0, 1])
    ax.set_ylim([2, 6])
    ax.set_xlabel("x")
    ax.set_ylabel("y")

plot_delta_line(axs[0], x, y, w=4, b=2, color="r")
plot_delta_line(axs[1], x, y, w=3, b=2, color="g")

plt.show()

Задача **поиска оптимальных параметров** модели сводится к задаче **поиска минимума функции потерь**.

## Поиск локального минимума

Посмотрим, как будет выглядеть ландшафт функции потерь для задачи линейной регрессии.

In [ ]:
w = np.arange(-10, 30, 1)
b = np.arange(-10, 10, 1)

w, b = np.meshgrid(w, b)

loss = np.zeros_like(w)
for i in range(w.shape[0]):
    for j in range(w.shape[1]):
        loss[i, j] = np.sum((y - (w[i, j] * x + b[i, j])) ** 2) / (len(x))

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
surf = ax.plot_surface(w, b, loss, cmap=plt.cm.RdYlGn_r, alpha=0.5)

ax.contourf(w, b, loss, zdir="z", offset=-1, cmap="RdYlGn_r", alpha=0.5)
ax.set_zlim(0, 20)

ax.set_xlabel("w")
ax.set_ylabel("b")
ax.set_title("MSE")

fig.colorbar(surf, location="left")
plt.show()

Необходимым (но недостаточным) условием локального минимума дифференцируемой функции является равенство нулю частных производных:

$$	\begin{equation*}
 \begin{cases}
   \displaystyle\frac{\partial \text{Loss}}{\partial w}=0,
   \\
   \displaystyle\frac{\partial \text{Loss}}{\partial b}=0.
 \end{cases}
\end{equation*} $$

Т.к. MSE для линейной регрессии — полином второй степени относительно $w$ и $b$, а полином второй степени не может иметь больше одного экстремума, то локальный минимум будет глобальным.

##  Метод наименьших квадратов

Реализуем простейшую модель линейной регрессии с использованием библиотеки NumPy на датасете, определённом выше.

Используем метод наименьших квадратов: [МНК, простейшие частные случаи 📚[wiki]](https://ru.wikipedia.org/wiki/Метод_наименьших_квадратов#Простейшие_частные_случаи).

$$w = \frac{n\sum_{i=1}^nx_iy_i - (\sum_{i=1}^nx_i)(\sum_{i=1}^ny_i)}{n\sum_{i=1}^nx_t^2 - (\sum_{i=1}^n x_t)^2};$$

$$b = \frac{\sum_{i=1}^ny_i - w(\sum_{i=1}^nx_i)}{n}.$$

По сути метод наименьших квадратов — это решение системы уравнений выше.

In [ ]:
def estimate_coef(x, y):
    n = len(x)
    w = (n * sum(np.multiply(x, y)) - sum(x) * sum(y)) / (
        n * sum(np.multiply(x, x)) - sum(x) ** 2
    )
    b = (sum(y) - w * sum(x)) / n
    return w, b


w, b = estimate_coef(x, y)

y_predicted = w * x + b

print(f"Estimated coefficients:\nb = {b[0]:.3f} \nw = {w[0]:.3f}")
print(f"Final equation: \ny = {w[0]:.3f}x +{b[0]:.3f}")

plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
plt.plot(x, y_predicted, color="g")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Полученное решение близко к исходной  зависимости $y=3\cdot x + 2$.

Для многомерного случая МНК можно записать [решение 📚[book]](https://sun.tsu.ru/mminfo/2016/Dombrovski/book/chapter-3/chapter-3-2.htm) в матричном виде:  
$$\vec w = (X^TX)^{-1}X^T\vec y$$
где $\vec w$ — вектор параметров модели, включающий $b$ (при записи этого решения используется трюк “столбец единиц”, о котором мы поговорим чуть позже),
$X$ — матрица входных признаков (с единичным столбцом),
$\vec y$ — вектор предсказываемых значений.



## Метрики регрессии

Вспомним метрики регрессии с предыдущей лекции.

В методе наименьших квадратов мы минимизируем $\text{MSE}$, эта метрика имеет [размерность 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D1%81%D1%82%D0%B8) квадрата размерности предсказываемого значения.

$$ \text{MSE}  = \frac{1}{N} \sum (y_i - f(x_i))^2$$

Чтобы получить оценку ошибки той же размерности, можно взять корень (root) от $\text{MSE}$. Это метрика $\text{RMSE}$:

$$ \text{RMSE} = \sqrt{\frac{1}{N} \sum (y_i - f(x_i))^2}$$

Или посчитать среднюю абсолютную ошибку $\text{MAE}$:

$$ \text{MAE} = \frac{1}{N} \sum |y_i - f(x_i)|$$



Также мы будем работать с $R^2$, которая принимает значения от $(-\inf, 1]$, где $1$  —  наилучший вариант. $R^2$  — [коэффициент детерминации 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D0%B4%D0%B5%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%B0%D1%86%D0%B8%D0%B8). Она характеризует долю дисперсии целевого значения, которую объясняет модель.

$$R^2 = 1 - \frac{\text{MSE}}{\sigma^2}=1 - \frac{\sum {(y_i-f(x_i))^2}}{\sum{(y_i-\bar{y})^2}},$$

$$\bar{y} = \frac{1}{N}\sum {y_i},$$

где $\sigma^2$ — дисперсия.

In [ ]:
from sklearn.metrics import (
    mean_squared_error,
    root_mean_squared_error,
    mean_absolute_error,
    r2_score,
)


def print_metrics(y_true, y_predicted):
    print(f"Mean squared error: {mean_squared_error(y_true, y_predicted):.3f}")
    print(
        "Root mean squared error: ",
        f"{root_mean_squared_error(y_true, y_predicted):.3f}",
    )
    print(f"Mean absolute error: {mean_absolute_error(y_true, y_predicted):.3f}")
    print(f"R2 score: {r2_score(y_true, y_predicted):.3f}")


print_metrics(y, y_predicted)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/linear_regression.png" width="600"></center>

<center><em>Когда $R^2$ около нуля, модель плохо объясняет данные</em></center>

<center><em>Source: <a href="https://xkcd.com/1725">https://xkcd.com/1725</a></em></center>

## Модель линейной регрессии из библиотеки scikit-learn

Свою линейную регрессию мы написали. Теперь поработаем с моделью из Sklearn.

Рассмотрим задачу предсказания успеваемости студента на основе информации о количестве потраченного им на изучение материала времени в часах.

Загрузим датасет [Students Score 🛠️[doc]](https://www.kaggle.com/datasets/shubham47/students-score-dataset-linear-regression). Датасет содержит два числовых значения — часы и результаты.

In [ ]:
import pandas as pd

dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/student_scores.csv"
)
print(dataset.shape)
dataset.head()

Построим график зависимости результата от затраченного времени, а также отобразим распределения каждой из переменных

In [ ]:
import seaborn as sns

sns.jointplot(data=dataset, x="Hours", y="Scores", height=5)
plt.show()

Разделим наши данные на train и test:

In [ ]:
from sklearn.model_selection import train_test_split

x = dataset.iloc[:, :-1].values  # column Hours
y = dataset.iloc[:, 1].values  # column Score

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

В случае мультимодального (несколько пиков) распределения значений целевой переменной в задаче регрессии может быть полезно задуматься о стратификации данных. Стратификация данных для задачи регрессии специфична и не реализована в Sklearn, о ней можно почитать в [Regression Analysis Based on Stratified Samples 🎓[article]](https://www.jstor.org/stable/2336525?seq=1), пример кода можно найти на [форуме ✏️[blog]](https://datascience.stackexchange.com/questions/33140/stratify-on-regression).

Теперь создадим модель для линейной регрессии. Чтобы не писать с нуля, воспользуемся готовой моделью из библиотеки Sklearn:

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

Обучим ее:

In [ ]:
regressor.fit(x_train, y_train)

Посмотрим, что получилось:

In [ ]:
x_train.shape

In [ ]:
x_points = np.linspace(min(x_train), max(x_train), 100)  # 100 dots at min to max
y_pred = regressor.predict(x_points)

plt.figure(figsize=(6, 4))
plt.plot(x_train, y_train, "o", label="Scores")
plt.plot(
    x_points,
    y_pred,
    label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_),
)
plt.title("Hours vs Percentage", size=12)
plt.xlabel("Hours Studied", size=12)
plt.ylabel("Percentage Score", size=12)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки:

In [ ]:
y_pred = regressor.predict(x_test)

x_points = np.linspace(min(x_test), max(x_test), 100)
y_pred = regressor.predict(x_points)

plt.figure(figsize=(6, 4))
plt.plot(x_test, y_test, "o", label="Scores")
plt.plot(
    x_points,
    y_pred,
    label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_),
)
plt.title("Hours vs Percentage", size=12)
plt.xlabel("Hours Studied", size=12)
plt.ylabel("Percentage Score", size=12)
plt.legend()
plt.show()

Выглядит неплохо.

Посчитаем метрики для наших значений:

In [ ]:
y_pred = regressor.predict(x_test)
print_metrics(y_test, y_pred)

Будьте осторожнее: модели отражают только те закономерности, которые видели в данных. Вероятность того, что студент, потративший на подготовку 20 часов, получит больше максимального балла, мала.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/extrapolating.png" width="500"></center>

<center><em>Source: <a href="https://xkcd.com/605">https://xkcd.com/605</a></em></center>





# Метод градиентного спуска

Мы решали задачу линейной регрессии аналитически (МНК), но это не всегда возможно по нескольким причинам:
* Для аналитического решения нужно считать обратную матрицу, это вычислительно сложно и матрица бывает плохо определенной.
* Данных может быть слишком много для того, чтобы их можно было одновременно положить в память для расчета обратной матрицы.
* Модели могут быть слишком сложными для поиска аналитического решения. Для сложных моделей ландшафт функции потерь может иметь рельеф с несколькими локальными минимумами. Например, при использовании более сложных функций потерь или в случае использования моделей, нелинейных по своим параметрам.

Давайте поговорим о том, что делать в таком случае.

## Градиент

Метод, который мы будем использовать, называется **“метод градиентного спуска”**. Для начала вспомним, что такое **градиент**. Возьмем функцию двух переменных:

$$\large f(x, y) = \sin(x\cdot y)$$

Она будет отличаться от функции потерь, которую мы визуализировали, тем, что у нее будет не один экстремум, а сложный рельеф. Рассчитаем ее на диапазоне значений от $0$ до $4$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

f = lambda x, y: np.sin(x * y)

x = np.linspace(0, 4, 1000)
y = np.linspace(0, 4, 1000)
xx, yy = np.meshgrid(x, y)
zz = f(xx, yy)

fig = plt.figure(figsize=(20, 7))


def show_3d(xx, yy, zz, fig):
    ax = fig.add_subplot(121, projection="3d")
    surf = ax.plot_surface(xx, yy, zz, cmap=plt.cm.RdYlGn_r)

    ax.contourf(xx, yy, zz, zdir="zz", offset=-2, cmap="RdYlGn_r")
    ax.set_zlim(-2, 2)

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("sin(xy)")
    fig.colorbar(surf, location="left")


show_3d(xx, yy, zz, fig)
plt.show()

Eсли $\varphi = \varphi(\vec{x})=\varphi(x_1 \dots x_n)$ — функция $n$ переменных, то её градиентом называется $n$-мерный вектор:
$$
\nabla \varphi(\vec{x})=
\begin{bmatrix}
\displaystyle\frac{\partial\varphi}{\partial x_1}\\
\displaystyle\frac{\partial\varphi}{\partial x_2}\\
...\\
\displaystyle\frac{\partial\varphi}{\partial x_n}\\
\end{bmatrix}
$$


Посчитаем градиент нашей функции $f(x, y)$. Для этого воспользуемся [**таблицей производных** 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%A2%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0_%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%BE%D0%B4%D0%BD%D1%8B%D1%85) и правилом вычисления [**производной сложной функции** 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D1%84%D1%84%D0%B5%D1%80%D0%B5%D0%BD%D1%86%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8) (Chain-rule):
$$\frac {\partial f} {\partial x} = \frac {\partial f} {\partial t} \cdot \frac {\partial t} {\partial x}$$

Это правило очень нам пригодится в будущем.

$$\nabla f(x, y)=\begin{bmatrix}
\displaystyle\frac{\partial f}{\partial x}\\
\displaystyle\frac{\partial f}{\partial y}\\
\end{bmatrix}
=\begin{bmatrix}
\displaystyle\frac{\partial\sin(xy)}{\partial(xy)}\cdot\frac{\partial(xy)}{\partial x}\\
\displaystyle\frac{\partial\sin(xy)}{\partial(xy)}\cdot\frac{\partial(xy)}{\partial y}\\
\end{bmatrix}
= \begin{bmatrix}
\cos(xy)\cdot y\\
\cos(xy)\cdot x\\
\end{bmatrix}
$$

Посчитаем градиент на том же диапазоне (сетка реже, т.к. мы будем рисовать не точки, а стрелочки):

In [ ]:
gradf = lambda x, y: (np.cos(x * y) * y, np.cos(x * y) * x)

xsmall = np.linspace(0, 4, 15)
ysmall = np.linspace(0, 4, 15)
xxsmall, yysmall = np.meshgrid(xsmall, ysmall)
gradx, grady = gradf(xxsmall, yysmall)

Так как **значение градиента в точке** — это вектор, мы можем говорить о его **величине** и **направлении**. Визуализируем наши расчеты: посмотрим на ландшафт функции $f(x, y)$ и направления градиентов.


In [ ]:
fig = plt.figure(figsize=(15, 5))
show_3d(xx, yy, zz, fig)

ax = fig.add_subplot(122)
ax.imshow(
    zz,
    extent=(np.min(x), np.max(x), np.min(y), np.max(y)),
    cmap="RdYlGn_r",
    origin="lower",
)
ax.set_xlabel("x")
ax.set_ylabel("y")

ax.quiver(xxsmall, yysmall, gradx, grady)
plt.show()

На рисунке выше значения градиента в точке обозначены чёрными стрелочками. Можно заметить, что длина стрелок в  районе максимальных и минимальных значений функции **почти нулевая**, стрелки направлены в направлении возрастания значения функции и наиболее длинные стрелки находятся в области наиболее резкого изменения значений функции.


Это проявление **свойств градиента**:
* Направление $\frac{\nabla f}{||\nabla f||}$ — сообщает нам направление максимального роста функции.

*  Величина $||\nabla f||$ — характеризует мгновенную скорость изменения значений функции.

## Идея градиентного спуска

Загрузим еще раз данные с зависимостью оценок студентов от времени подготовки.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


dataset = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/student_scores.csv"
)

x = dataset.iloc[:, :-1].values  # column Hours
y = dataset.iloc[:, 1].values  # column Score

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

Скачаем **код для интерактивной визуализации**. Он нужен только для объяснения и **не пригодится вам в работе**. Его разбирать мы не будем. Eсли интересно, можно изучить самостоятельно.

In [ ]:
# @title *Code for interactive visual
# source: https://github.com/TomasBeuzen/deep-learning-with-pytorch

!wget -qN https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/interactive_visualization.py

Для простоты рассмотрим одномерный случай. Будем подбирать только $w$, значение $b$ зафиксируем на уровне $2.83$. Визуализируем ошибку и значения $\dfrac{\partial \text{Loss}}{\partial w}$ для MSE Loss.

In [ ]:
from interactive_visualization import plot_grid_search
from sklearn.metrics import mean_squared_error


slopes = np.arange(5, 15, 0.5)
prediction = {f"{w}": w * x_train[:, 0] + 2.83 for w in slopes}
mse = np.array([mean_squared_error(y_train, w * x_train[:, 0] + 2.83) for w in slopes])
dmse_dw = np.array(
    [(2 * x_train[:, 0] * (w * x_train[:, 0] + 2.83 - y_train)).mean() for w in slopes]
)
plot_grid_search(x_train[:, 0], y_train, slopes, prediction, mse, dmse_dw)

 Видно, что оптимальное значение наклона соответствует минимуму MSE и нулю частной производной $\dfrac{\partial \text{Loss}}{\partial w}$. Аналогично будет, если мы возьмем в качестве Loss MAE.

In [ ]:
from sklearn.metrics import mean_absolute_error


slopes = np.arange(5, 15, 0.5)
prediction = {f"{w}": w * x_train[:, 0] + 2.83 for w in slopes}
mae = np.array([mean_absolute_error(y_train, w * x_train[:, 0] + 2.83) for w in slopes])
dmae_dw = np.array(
    [
        (x_train[:, 0] * np.sign(w * x_train[:, 0] + 2.83 - y_train)).mean()
        for w in slopes
    ]
)
plot_grid_search(x_train[:, 0], y_train, slopes, prediction, mae, dmae_dw)

Итого, т.к. градиент указывает направление наибольшего возрастания функции:

* если $\dfrac{\partial \text{Loss}}{\partial w} < 0$, то нам имеет смысл “идти” в сторону возрастания $\dfrac{\partial \text{Loss}}{\partial w}$;

* если $\dfrac{\partial \text{Loss}}{\partial w} > 0$ — в сторону убывания.

**Метод градиентного спуска** — итеративный метод, идея которого заключается в том, чтобы небольшими шажками “идти” в **обратную от градиента сторону**:

$$\large \vec w_{n+1} = \vec w_{n} - α \cdot \nabla_{\vec w_{n}} \text{Loss},$$
где $α$ — скорость обучения.


Попробуем реализовать это в коде (для простоты только для $w$ при $b=2.83$).

Обратите внимание:
- предсказание считается `w * x_test + b`
- градиент считается `2 * (x * (w * x + b - y)).mean()`

Через пепемножение скалярных величин.

In [ ]:
def gradient(x, y, w, b):
    return 2 * (x * (w * x + b - y)).mean()


def gradient_descent(x_train, y_train, x_test, y_test, w, alpha, b=2.83, iteration=10):
    """Gradient descent for optimizing slope in simple linear regression"""
    # history
    ws = [w]
    mse_train = [mean_squared_error(y_train, w * x_train + b)]
    dmse_train = []
    mse_test = [mean_squared_error(y_test, w * x_test + b)]
    prediction = {w: w * x_train + b}
    print(
        f"Iteration 0: w = {w:.2f}, Loss_train = {mse_train[0]:.2f}, "
        f"Loss_test = {mse_test[0]:.2f}."
    )
    for i in range(iteration):
        # adjust w based on gradient * learning rate
        grad = gradient(x_train, y_train, w, b)
        w -= alpha * grad  # adjust w based on gradient * learning rate
        # history
        ws.append(w)
        mse_train.append(mean_squared_error(y_train, w * x_train + b))
        dmse_train.append(grad)
        mse_test.append(mean_squared_error(y_test, w * x_test + b))
        prediction[w] = w * x_train + b
        print(
            f"Iteration {i+1}: w = {w:.2f}, Loss_train = {mse_train[i]:.2f}, "
            f"Loss_test = {mse_test[i]:3.2f}."
        )
    return ws, prediction, mse_train, dmse_train, mse_test

Обучим нашу модель:

In [ ]:
slopes, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train[:, 0], y_train, x_test[:, 0], y_test, w=5, alpha=0.01, iteration=7
)

Визуализируем процесс обучения:

In [ ]:
from interactive_visualization import plot_gradient_descent

plot_gradient_descent(x_train[:, 0], y_train, slopes, prediction, mse_train, dmse_train)

Видно, что за 7 эпох мы получили то же значение $w$, что получали при использовании `LinearRegression`. При этом мы пришли в минимум MSE и ноль градиента.

"Эпохой" обучения мы будем называть последовательность шагов оптимизации, при которой мы использовали информацию обо всех данных из тренировочной выборки. В нашем примере мы использовали всю обучающую выборку на каждом шаге градиентного спуска, потому "шаги градиентного спуска" и "эпохи" в данном примере синонимичны.


В реальности мы будем работать с функциями многих переменных, поэтому смотреть на сходимость по одной переменной — не самый оптимальный вариант. Более эффективно будет посмотреть на зависимость Loss от количества эпох для train и test.

In [ ]:
def plot_mse(mse_train, mse_test):
    plt.figure(figsize=(10, 4))
    plt.title("Learning curve")
    plt.plot(mse_train, label="train")
    plt.plot(mse_test, label="test")
    plt.legend()

    plt.xlabel("iterations", fontsize=12)
    plt.ylabel("MSE Loss", fontsize=12)

    plt.grid(True)
    plt.show()

Такие графики называют кривыми обучения. Посмотрим на кривые обучения при нашей скорости обучения $α=0.01$.

In [ ]:
plot_mse(mse_train, mse_test)

Видно, что **Loss падает** как на **train**, так и на **test** выборке. Также мы можем сказать, что **сеть обучилась**: train и test **графики вышли на плато**. При этом не произошло **переобучение**: ошибка на **test** выборке **не начала расти** (про переобучение поговорим позже).

В полученных графиках есть особенность, которая бросается в глаза опытному в обучении моделей человеку: **Loss на test выборке меньше, чем на train**. Это показатель того, что **с данными что-то не так**. Так бывает при утечке данных (об утечке данных вы подробнее узнаете в следующих лекциях), но в данном случае, test выборка просто слишком мала, чтобы отражать генеральную совокупность (всего 5 студентов).

## Выбор скорости обучения

**Скорость (шаг) обучения** — некоторый коэффициент, как правило, небольшой, который позволяет нам управлять размером шага. У нас есть локальный минимум, в который мы хотим попасть. Если мы сделаем слишком большой шаг, то мы его перескочим (график справа). Нужно подобрать шаг, который не позволит перескочить минимум, но в то же время такой, чтобы тот же процесс не шел слишком медленно (как на графике слева).

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/learning_rate_optimal_value.png">

Посмотрим на скорость обучения на нашем примере. При **маленькой скорости обучения** мы будем очень медленно сходиться к минимуму.

In [ ]:
slopes, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train[:, 0], y_train, x_test[:, 0], y_test, w=5, alpha=0.0005, iteration=30
)

In [ ]:
plot_gradient_descent(x_train[:, 0], y_train, slopes, prediction, mse_train, dmse_train)

Спустя 30 итераций оранжевая прямая плохо отражает генеральную совокупность. Мы не достигли минимума MSE и нуля градиента.

Посмотрим, как выглядят кривые обучения:

In [ ]:
plot_mse(mse_train, mse_test)

Модель недообучена — значения Loss не вышли на плато.

Посмотрим на **достаточно большую скорость обучения**.

In [ ]:
slopes, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train[:, 0], y_train, x_test[:, 0], y_test, w=5, alpha=0.027, iteration=15
)

In [ ]:
plot_gradient_descent(x_train[:, 0], y_train, slopes, prediction, mse_train, dmse_train)

Шаг, получаемый умножением градиента на скорость обучения, получается достаточно большим, чтобы “перескочить” локальный минимум, но при этом модель все-таки попадает в него. Кривые обучения при этом успешно выходят на плато.

In [ ]:
plot_mse(mse_train, mse_test)

В финале посмотрим на **очень большую скорость обучения**.

In [ ]:
slopes, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train[:, 0],
    y_train,
    x_test[:, 0],
    y_test,
    w=5,
    alpha=0.034,
    iteration=5,
)

Шаг, который мы делаем, слишком большой. Мы не попадаем в локальный минимум.

In [ ]:
plot_gradient_descent(x_train[:, 0], y_train, slopes, prediction, mse_train, dmse_train)


По кривым обучения видно, что модель не сошлась: ошибка растет.


In [ ]:
plot_mse(mse_train, mse_test)

Выбор скорости обучения будет зависеть от модели и данных. В 7 лекции вы познакомитесь с различными модификациями метода градиентного спуска и узнаете больше о выборе скорости обучения, а пока ориентируйтесь на кривые обучения.

## Единый подход к учету смещения

Пока мы настраивали только одну переменную, но даже в случае предсказания оценки по времени подготовки у нас две переменные: вес $w$ и смещение $b$.

Когда признаков станет больше, у нас получится “лапша” из слагаемых:
$$y = b + w_1\cdot x_1 + w_2\cdot x_2 + w_3\cdot x_3 + w_4\cdot x_4 + w_5\cdot x_5 + ... + w_n\cdot x_n$$

Нам бы хотелось записать их компактно, чтобы не усложнять код и использовать один и тот же код для данных с разным количеством признаков.  Для этого мы будем использовать матричное перемножение и трюк **“столбец единиц”**, который реализует **единый подход к учету смещения**.

Обозначим вектор-столбец из настраиваемых параметров:
$$\vec w = \begin{bmatrix}
b \\ w \\
\end{bmatrix}$$


In [ ]:
w = np.array([[0.5], [5]])
w

К матрице (в нашем случае был только один признак, поэтому у нас будет вектор-столбец) признаков слева "дорисуем" столбец единиц:
$$X = \begin{bmatrix}
1 & X \\
\end{bmatrix} =
\begin{bmatrix}
1 & 2.7 \\
1 & 3.3 \\
... & ...\\
1 & 9.2 \\
\end{bmatrix}$$

**Предупреждение:** добавлять столбец единиц нужно, только если вы сами пишете модель. **Если вы пользуетесь готовыми моделями, в этом нет необходимости.**


In [ ]:
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
x_test

Матрицу $X$ можно матрично перемножить со столбцом $\vec w$, т.к количество столбцов $X$ совпадает с количеством строк в $\vec w$:

In [ ]:
x_train.shape, w.shape

В общем случае:

$$\large \vec y = b + w_1\cdot x_1 + w_2\cdot x_2 + w_3\cdot x_3 + w_4\cdot x_4 + w_5\cdot x_5 + ... + w_n\cdot x_n = X\vec w $$  

Эту формулу можно свести к нескольким символам кода (`@` — матричное умножение):


In [ ]:
y_pred = x_test @ w
y_pred

## Необходимость нормализации

Реализуем многомерный градиентный спуск

In [ ]:
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

Обратите внимание:
- предсказание считается `x_train @ w`
- градиент считается `2 * (x.T @ (x @ w) - x.T @ y) / len(x)`

Через перемножение матриц!

In [ ]:
def gradient(x, y, w):
    """Gradient of mean squared error."""
    return 2 * (x.T @ (x @ w) - x.T @ y) / len(x)


def gradient_descent(x_train, y_train, x_test, y_test, w, alpha, iteration=10):
    """Gradient descent for optimizing slope in simple linear regression"""
    # history
    ws = np.zeros((iteration + 1, 2))
    ws[0] = w[:, 0]
    mse_train = [mean_squared_error(y_train, x_train @ w)]
    dmse_train = []
    mse_test = [mean_squared_error(y_test, x_test @ w)]
    prediction = {(w[0][0], w[1][0]): x_train @ w}

    print(
        f"Iteration 0: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
        f"Loss_train = {mse_train[0]:.2f}, "
        f"Loss_test = {mse_test[0]:.2f}."
    )

    for i in range(iteration):
        # adjust w based on gradient * learning rate
        grad = gradient(x_train, y_train, w)
        w -= alpha * grad  # adjust w based on gradient * learning rate
        # history
        ws[i + 1] = w[:, 0]
        mse_train.append(mean_squared_error(y_train, x_train @ w))
        dmse_train.append(grad)
        mse_test.append(mean_squared_error(y_test, x_test @ w))
        prediction[(w[0][0], w[1][0])] = x_train @ w

        print(
            f"Iteration {i+1}: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
            f"Loss_train = {mse_train[i]:.2f}, "
            f"Loss_test = {mse_test[i]:3.2f}."
        )
    return ws, prediction, mse_train, dmse_train, mse_test

Попробуем обучить модель:

In [ ]:
w = np.array([[0.5], [5]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train,
    y_train,
    x_test,
    y_test,
    w,
    0.01,
)

Мы не дошли до оптимальной прямой $y = 9.68x+2.83$, которую вычисляли выше.

При этом график Loss выглядит неплохо:

In [ ]:
plot_mse(mse_train, mse_test)

Такое поведение связано с ландшафтом функции потерь: значение ошибки по оси $b$ изменяется намного медленнее, чем по оси $w$.

In [ ]:
from interactive_visualization import plot_grid_search_2d

intercepts = np.arange(-7.5, 12.5, 0.1)  # b
slopes = np.arange(5, 15, 0.1)  # w
plot_grid_search_2d(x_train[:, 1], y_train, slopes, intercepts)

Поэтому основное изменение значений происходит вдоль оси $w$, а $b$ меняется слабо (значение $b$ далеко от ожидаемого).

In [ ]:
from interactive_visualization import plot_gradient_descent_2d

plot_gradient_descent_2d(
    x_train[:, 1],
    y_train[:, 0],
    ws,
    slopes,
    intercepts,
)

Чтобы исправить ситуацию, применим `StandardScaler`:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(np.expand_dims(x_train[:, 1], axis=1)).flatten()
x_test_scaled = scaler.transform(np.expand_dims(x_test[:, 1], axis=1)).flatten()

In [ ]:
intercepts = np.arange(40, 60, 0.1)  # b
slopes = np.arange(15, 35, 0.1)  # w

plot_grid_search_2d(x_train_scaled, y_train, slopes, intercepts)

In [ ]:
x_train_scaled = np.hstack(
    (np.ones((len(x_train_scaled), 1)), np.expand_dims(x_train_scaled, axis=1)),
)

x_test_scaled = np.hstack(
    (np.ones((len(x_test_scaled), 1)), np.expand_dims(x_test_scaled, axis=1)),
)

Т.к. диапазоны $x$ изменились, значения $w$ и $b$ тоже изменятся.


In [ ]:
w = np.array([[57.0], [33.0]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train_scaled, y_train, x_test_scaled, y_test, w, 0.35, iteration=10
)

In [ ]:
plot_mse(mse_train, mse_test)

Проверим, что после нормализации мы сходимся к $y = 9.68x + 2.83$.  Для этого используем данные о матожидании и дисперсии из `StandardScaler`.

In [ ]:
b = ws[-1][0] - ws[-1][1] * scaler.mean_ / (scaler.var_) ** 0.5
w = ws[-1][1] / (scaler.var_) ** 0.5

print(f"y = {w[0]:.2f}x + {b[0]:.2f}")

По визуализации видно, что $w$ и $b$ изменяются во время обучения.

In [ ]:
plot_gradient_descent_2d(
    x_train_scaled[:, 1],
    y_train[:, 0],
    ws,
    slopes,
    intercepts,
)

## Cтохастический градиентный спуск

[[blog] ✏️ Пост о стохастическом градиентом спуске](https://www.tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter2_stochastic-gradient-descent.html)

До этого мы обучали модель, рассчитывая градиент по **всей train выборке**. То есть один шаг градиентного спуска мог быть "долгим" и **вычислительно сложным** из-за необходимости определения градиента функции потерь на всех потенциально многочисленных train объектах. Можем ли мы совершать шаги градиентного спуска "чаще" и существенно ускорить сходимость модели к верному ответу?


Поэтому появляется идея **стохастического градиентного спуска**: мы можем делать шаг обучения, рассчитывая градиент не по всей выборке (**batch**), а по нескольким случайно выбранным объектам (**mini-batch**) или даже по одному случайно выбранному объекту (**stochastic**).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/define_size_of_batch.png" width="500"></center>

Можно [показать 📚[book]](https://academy.yandex.ru/handbook/ml/article/shodimost-sgd), что **стохастический градиентный спуск сходится к минимуму** (глобальному или локальному) функции потерь, как и "обычный" градиентный спуск, при меньшем объеме вычислений.
Важным условием является случайный выбор объектов. Если мы будем использовать одну и ту же последовательность выборок, это приведет к накоплению ошибки и смещению результата.

Добавим создание подвыборки к нашему алгоритму:

In [ ]:
def stochastic_gradient_descent(
    x_train,
    y_train,
    x_test,
    y_test,
    w,
    alpha,
    iteration=10,
    batch_size=None,
):
    """Gradient descent for optimizing slope in simple linear regression"""
    # history
    ws = np.zeros((iteration + 1, 2))
    ws[0] = w[:, 0]
    mse_train = [mean_squared_error(y_train, x_train @ w)]
    dmse_train = []
    mse_test = [mean_squared_error(y_test, x_test @ w)]
    prediction = {(w[0][0], w[1][0]): x_train @ w}

    print(
        f"Iteration 0: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
        f"Loss_train = {mse_train[0]:.2f}, "
        f"Loss_test = {mse_test[0]:.2f}."
    )

    for i in range(iteration):
        if not batch_size:
            x_sample = x_train
            y_sample = y_train
        else:
            indxs = np.random.choice(x_train.shape[0], batch_size)
            x_sample = x_train[indxs, :]
            y_sample = y_train[indxs, :]

        # adjust w based on gradient * learning rate
        grad = gradient(x_sample, y_sample, w)
        w -= alpha * grad  # adjust w based on gradient * learning rate
        # history
        ws[i + 1] = w[:, 0]
        mse_train.append(mean_squared_error(y_train, x_train @ w))
        dmse_train.append(grad)
        mse_test.append(mean_squared_error(y_test, x_test @ w))
        prediction[(w[0][0], w[1][0])] = x_train @ w
        if (i + 1) % 10 == 0:
            print(
                f"Iteration {i+1}: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
                f"Loss_train = {mse_train[i]:.2f}, "
                f"Loss_test = {mse_test[i]:3.2f}."
            )
    return ws, prediction, mse_train, dmse_train, mse_test

Чтобы сравнить результаты, будем использовать одни и те же количество итераций и скорость обучения. Чтобы компенсировать стохастичность, возьмем маленькое значение $\alpha$ и $100$ итераций.

Для всего train сета мы посчитаем градиент для $20\cdot100 = 2000$ точек.

In [ ]:
w = np.array([[57.0], [33.0]])
ws, prediction, mse_train, dmse_train, mse_test = stochastic_gradient_descent(
    x_train_scaled,
    y_train,
    x_test_scaled,
    y_test,
    w,
    0.02,
    iteration=100,
    batch_size=None,
)

f1 = plot_gradient_descent_2d(
    x_train_scaled[:, 1],
    y_train[:, 0],
    ws,
    slopes,
    intercepts,
    mode="lines",
    title="Batch gradient descent",
)

Для стохастического градиентного спуска (размер $\text{batch}=1$) мы посчитаем градиент для $1\cdot100 = 100$ точек.

In [ ]:
np.random.seed(42)
w = np.array([[57.0], [33.0]])
ws_stohastic, prediction, mse_train, dmse_train, mse_test = stochastic_gradient_descent(
    x_train_scaled,
    y_train,
    x_test_scaled,
    y_test,
    w,
    0.02,
    iteration=100,
    batch_size=1,
)
f2 = plot_gradient_descent_2d(
    x_train_scaled[:, 1],
    y_train[:, 0],
    ws_stohastic,
    slopes,
    intercepts,
    mode="lines",
    title="Stochastic gradient descent",
)

Для стохастического спуска с $\text{mini-batch}=5$ мы посчитаем градиент для $5\cdot100=500$ точек.

In [ ]:
np.random.seed(42)
w = np.array([[57.0], [33.0]])
(
    ws_mini_batch,
    prediction,
    mse_train,
    dmse_train,
    mse_test,
) = stochastic_gradient_descent(
    x_train_scaled,
    y_train,
    x_test_scaled,
    y_test,
    w,
    0.02,
    iteration=100,
    batch_size=5,
)
f3 = plot_gradient_descent_2d(
    x_train_scaled[:, 1],
    y_train[:, 0],
    ws_mini_batch,
    slopes,
    intercepts,
    mode="lines",
    title="Mini-batch gradient descent",
)

Мы ищем минимум для всех данных.
- Градиент, рассчитанный по одному объекту, будет специфичен. Трек обучения  в случае стохастического градиентного спуска будет запутанным, а итоговая ошибка будет расти с увеличением скорости обучения (мы взяли низкую скорость).
- Градиент, рассчитанный по нескольким объектам будет давать лучшую оценку градиента для всех данных. Трек будет менее сложным.
- Градиент, рассчитанный по всей выборке, будет давать наиболее точное направление (при отсутствии "шума" в обучающих данных).

In [ ]:
from interactive_visualization import plot_panel

plot_panel(f1, f2, f3)

Для **ускорения расчетов** рекомендовано использовать **максимальный размер mini-batch**, который помещается в память, но это не всегда дает лучший результат. На 7 лекции вы увидите, что для сложных моделей стохастичность, связанная с небольшим размером батча, может помочь выбраться из локального минимума и найти более глубокий.


# Условия применимости линейной регрессии

## Квартет Энскомбе (Anscombe’s quartet)

Мы применили модель линейной регрессии, посчитали метрики и решили что наша модель достаточно хороша. Но так ли это на самом деле?

В 1973 году Фрэнсис Энскомбе тоже [задумался [art] 🎓](https://www.jstor.org/stable/2682899) над этим вопросом. Он предложил 4 набора данных с одинаковыми статистиками, которые показали некоторые типичные ошибки, возникающие при решении регрессионных задач. Посмотрим на эти наборы данных и мы (для простоты тут нет разделения на train/test).

Для начала посчитаем статистики набора данных, коэффиценты линейной модели и коэффициент детерминации:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

df = sns.load_dataset("anscombe")

grouped = df.groupby("dataset")

summary_results = pd.DataFrame(
    columns=[
        "mean_x",
        "mean_y",
        "std_x",
        "std_y",
        "correlation",
        "slope",
        "offset",
        "R2",
        "MSE",
        "MAE",
    ]
)

for key in grouped.groups.keys():
    data = grouped.get_group(key)
    regressor = LinearRegression()
    regressor.fit(data["x"].values.reshape(-1, 1), data["y"].values)
    slope = regressor.coef_[0]
    offset = regressor.intercept_
    y_pred = regressor.predict(data["x"].values.reshape(-1, 1))
    r2 = r2_score(data["y"].values, y_pred)
    mse = mean_squared_error(data["y"].values, y_pred)
    mae = mean_absolute_error(data["y"].values, y_pred)
    summary_results.loc[key] = (
        grouped.mean().loc[key]["x"],
        np.round(grouped.mean().loc[key]["y"], 2),
        np.round(grouped.std().loc[key]["x"], 5),
        np.round(grouped.std().loc[key]["y"], 2),
        np.round(grouped.corr().loc[(key, "x")]["y"], 3),
        np.round(slope, 3),
        np.round(offset, 2),
        np.round(r2, 2),
        np.round(mse, 2),
        np.round(mae, 2),
    )

summary_results

Видно, что по статистикам наборы данных почти идентичны. Посмотрим на сами данные:

In [ ]:
import matplotlib.pyplot as plt

sns.lmplot(
    x="x",
    y="y",
    col="dataset",
    hue="dataset",
    data=df,
    col_wrap=2,
    ci=None,
    palette="muted",
    height=4,
    scatter_kws={"s": 50, "alpha": 1},
)
plt.gcf().suptitle("Anscombe's Quartet", x=0.5, y=1.02)
plt.tight_layout()
plt.show()

Что мы видим:
- Набор данных “I” - подходит для линейной регрессии,
- Набор данных “II” - визуализирует ошибку спецификации. Мы предположили, что $y$ линейно зависит от $x$, а на самом деле зависимость квадратичная.
- В наборе данных “III” присутствует выброс, который приводит к смещенной оценке. Нужно разобраться с природой выброса: если это ошибка в данных - удалить, если это реальное значение - необходимо расширение выборки и применение более сложной модели.
- Набор данных  “IV” - плохо подходит для регрессии: 9 из 10 значений $x$ совпадают и дают разные значения $y$, что противоречит предположению, что $y = f(x)$.

Эти проблемы легко визуализировать и заметить, если у нас только 1 признак, но что делать когда признаков много?

 ## Анализ остатков

Тут нам нужно познакомиться с [теоремой Гаусса-Маркова](https://ru.wikipedia.org/wiki/Теорема_Гаусса_—_Маркова), согласно которой метод наименьших квадратов дает оптимальную несмещенную оценку $y$, если:
* модель данных правильно специфицированна (зависимость действительно линейная), спецификация модели:
$$y_i = b+w_1·x_{i1}+w_2⋅x_{i2}+...+w_k⋅x_{ik}+ϵ_i$$
где $(X_i, y_i)$ - пары признаков и целевых значений $i\in(1, 2, ..., n)$,
$b$ и $w_i$ - параметры модели, $ϵ_i$ - шум в определении $y_i$
* все выборки $X_i$ детерминированы и не все равны между собой,
* признаки линейно независимы,
* ошибки не носят систематического характера, а именно гомоскедастичность (нулевое матожидание и одинаковая дисперсия ошибок для всех $i$) и отсуствие автокорреляции.



В условиях этой много говориться об ошибках. Давайте визуализируем ошибки/невязки/остатки $\delta y = y_{true}-y_{pred}$ для квартета.

In [ ]:
import matplotlib.image as mpimg

fig, axs = plt.subplots(figsize=(8, 8), nrows=2, ncols=2)
fig.tight_layout(pad=-4.0, w_pad=-3.5, h_pad=-6.0)

for key, ax in zip(grouped.groups.keys(), [axs[0][0], axs[0][1], axs[1][0], axs[1][1]]):
    data = grouped.get_group(key)
    regressor = LinearRegression()
    regressor.fit(data["x"].values.reshape(-1, 1), data["y"].values)
    y_pred = regressor.predict(data["x"].values.reshape(-1, 1))
    df = {
        "x": data["x"].values,
        "y": data["y"].values,
        "dy": data["y"].values - y_pred,
    }
    df = pd.DataFrame(df)
    g = sns.JointGrid(
        data=df,
        x="y",  # 'x'
        y="dy",
        height=4,
        ratio=4,
    )
    g.refline(y=0, color="red")
    g.plot_joint(sns.scatterplot)
    g.plot_marginals(sns.kdeplot)
    g.ax_marg_x.remove()
    g.savefig("g.png")
    plt.close(g.fig)
    ax.imshow(mpimg.imread("g.png"))
    ax.set_axis_off()
    ax.set_title(key)

Что мы видим: остатки  набора “I” распределены случайно, в то время, как остатки набора “II”, “III” и “IV” - подчиняются некоторой системе. **Вывод: визуализируя остатки можно сделать вывод о применимости линейной регресси.**


Мы посмотрели на корреляцию остатков, кроме нее есть ограничение на **гомоскедастичность** (нулевое матожидание и одинаковая дисперсия ошибок для всех подвыборок). Попробуем применить линейную регрессию к данным построенным по правилу $y=10+0.5x$, дисперсия ошибки будет расти с $x$.

In [ ]:
import numpy as np

rng = np.random.RandomState(42)
x = np.linspace(start=0, stop=10, num=40)
y_true = 10 + 0.5 * x
y = y_true + rng.normal(loc=0, scale=0.1 + 0.1 * x, size=x.shape[0])

regressor = LinearRegression()
regressor.fit(x[:, np.newaxis], y)
y_pred = regressor.predict(x[:, np.newaxis])

fig, axs = plt.subplots(figsize=(9, 4), nrows=1, ncols=2)
fig.tight_layout(pad=4.0)

axs[0].scatter(x, y)
axs[0].plot(x, y_pred, label="y_pred", color="red")
axs[0].plot(x, y_true, label="y_true", color="green")
axs[0].set_title("Regression")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")
axs[0].legend()

axs[1].scatter(y, y - y_pred)
axs[1].axhline(y=0, color="red")
axs[1].set_title("Residuals")
axs[1].set_xlabel("y")
axs[1].set_ylabel("dy")

plt.show()


Видим, что предсказание не совсем корректное. Это связано с вкладом больших ошибок. В таком случае имеет смысл посмотреть в сторону [взвешенного метода наименьших квадратов](https://stackoverflow.com/questions/35236836/weighted-linear-regression-with-scikit-learn) или подумать над [стабилизирующим преобразованием](https://ajeyvenkataraman.wordpress.com/2020/03/04/dealing-with-heteroscedasticity-in-python/).


Подробнее про анализ остатков можно почитать в [книге](https://ema.drwhy.ai/residualDiagnostic.html).

## Проблема корреляции признаков

Часто может оказаться, что признаковое описание объекта избыточно и между различными признаками имеются связи. Для устойчивости работы линейных моделей важно, чтобы среди признаков не было линейных связей (скоррелированных пар).

Например, если мы будем решать задачу регрессии на наборе признаков $x_1 \dots x_n$, среди которых есть связь $x_2 = 5 x_1$, и возьмём линейную модель вида
$$\large y = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b,$$
то с учётом данной связи мы можем записать:
$$\large y = w_1 x_1 + w_2 (5x_1) + \dots + w_n x_n + b = (w_1 + 5 w_2) x_1 +  w_3 x_3 + \dots + w_n x_n + b.$$

Таким образом, наша модель теперь учитывает признак $x_1$ с одним "общим" весом $(w_1 + 5 w_2)$, несмотря на то, что он закодирован двумя независимыми параметрами. Решение, то есть набор весовых коэффициентов $w_i$, перестало быть единственным, так как мы теперь можем делать произвольные преобразования с числами $w_1$ и $w_2$ до тех пор, пока $(w_1 + 5 w_2)$ остаётся неизменным:

$$(w_1 + 5 w_2) = \{w_1 \rightarrow  w_1 + 5000 ,\, w_2 \rightarrow  w_2 - 1000 \} $$
$$(w_1 + 5 w_2) = \{w_1 \rightarrow  w_1 + 5000000 ,\, w_2 \rightarrow  w_2 - 1000000 \} $$
$$(w_1 + 5 w_2) = \{w_1 \rightarrow  w_1 + 5000000000 ,\, w_2 \rightarrow  w_2 - 1000000000 \} $$
$$(w_1 + 5 w_2) = \{w_1 \rightarrow  w_1 + 5000000000000 ,\, w_2 \rightarrow  w_2 - 1000000000000 \} $$
$$(w_1 + 5 w_2) = \{w_1 \rightarrow  w_1 + Nan ,\, w_2 \rightarrow  w_2 + Nan\}$$

**Чем это плохо?**

В случае корреляции признаков задача не имеет единственного решения и не существует обратной матрицы, обеспечивающей аналитическое решение. Мы можем использовать градиентные методы для поиска решения, но
при этом веса модели могут неконтролируемо расти. При этом **суммарный вклад** признаков может быть **мал**.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/carrelation_problem1.png" width="700"></center>

Мы можем оценивать **важность признаков в линейной модели**, используя **веса** перед ними (признаки должны быть нормализованы). Чем больше модуль веса, тем больше вклад. Для коррелированных признаков важность будет переоценена.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/carrelation_problem2.png" width="700"></center>

Кроме того, нужно помнить, что диапазоны числовых переменных ограничены. При неконтролируемом росте весов значение может выйти за диапазон и превратиться в ~~тыкву~~ `Nan`.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/carrelation_problem3.png" width="700"></center>

**Что делать?**

Загрузим **датасет Diabetes** ([Diabetes dataset 🛠️[doc]](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset)), в котором таргетом является число, характеризующее стадию заболевания, а на признаками явдяются пол, возраст, иднекс массы тела и результаты анализа крови.

In [ ]:
# load dataset
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

x = diabetes.data  # features
y = diabetes.target  # target
print(f"x shape: {x.shape}, y shape: {y.shape}")
print(f"x[0]: \n {x[0]}")
print(f"y[0]: \n {y[0]}")

Быстрее и удобнее можно посмотреть на данные, используя pandas. К тому же Colab добавил возможность визуализации данных (для этого можно тыкнуть синий значок диаграммы ▆ █ ▄  справа от таблицы):

In [ ]:
import pandas as pd

diabetes_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
diabetes_df

Colab добавил возможность визуализации данных (для этого можно тыкнуть синий значок диаграммы ▆ █ ▄  справа от таблицы). Colab делает не полную визуализацию признаков, но и на данных изображениях можно найти полезную **информацию о выбросах** (из графика **Values**), **плотности распределений** (из графика **Distributions**) и о наличии **зависимости между переменными** (из графика **2-d distributions**).

Кроме того, можно тыкнуть на рисунок, посмотреть и скопировать код визуализации, чтобы применить к другим данным или изменить под свои нужды. Так нажав на **2-d distributions**, скопировав код и поменяв названия признаков, мы можем увидеть, что значения признаков *s1* и *s2* имеют зависимость, близкую к линейной, что не очень хорошо (почему, обсудим позже).


In [ ]:
from matplotlib import pyplot as plt

diabetes_df.plot(kind="scatter", x="s1", y="s2", s=32, alpha=0.8)
plt.gca().spines[
    [
        "top",
        "right",
    ]
].set_visible(False)

Теперь давайте посмотрим на сами данные. У нас есть 441 строк, в каждой из которой по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд).

In [ ]:
diabetes_df.describe()

Данные немного странно нормализованы (они нормализованны так, чтобы сумма квадратов значений в каждом столбце была единица), для линейной модели - это не очень хорошо, применим StandartScaler.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

print(y_val.shape)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

x_train_norm = scaler_x.fit_transform(x_train)
y_train_norm = scaler_y.fit_transform(y_train[:, np.newaxis])[:, 0]

x_val_norm = scaler_x.transform(x_val)
y_val_norm = scaler_y.transform(y_val[:, np.newaxis])[:, 0]

diabetes_train_norm = pd.DataFrame(x_train_norm, columns=diabetes.feature_names)
diabetes_train_norm.describe()

Визуализируем линейные зависимости между признаками при помощи построения матрицы попарных корреляций.

In [ ]:
import numpy as np

# Compute the correlation matrix
corr = diabetes_train_norm.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8, 6))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    vmin=-1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)
plt.show()

Как мы уже видели наших данных сильно скоррелированы признаки S1 и S2.

В случае корреляции можно:
- **добавить регуляризацию**;
- **оставить один признак**;
- если есть вероятность, что при удалении признаков часть информации будет потеряна, можно оставить **один признак** неизменным и вычесть его из остальных (оставить только **разницу**). Неинформативные шумовые признаки можно удалить (как это делать, вы узнаете на 4-й лекции).


## Регуляризация


###L2 vs L1

Мы можем ограничить значения весов, введя в функционал ошибок
специальную добавку, называемую регуляризацией. L2-регуляризация имеет формулу:


$$\large L_2 = \alpha \sum_i w_i^2,$$
где $\alpha$ — это коэффициент регуляризации.


 Введение **L2-регуляризации** приводит к тому, что **большие веса больше штрафуются** и предпочтение отдается решениям, использующим **малые значения весов**. При этом модель будет **сохранять скоррелированные и неважные признаки с маленькими весами**.

Это связано с градиентом $L_2$:
$$\large L'_{2w_i} = 2\alpha w_i$$
Он будет “тянуть” модель в сторону большого количества маленьких весов.

Для отбора признаков можно использовать L1-регуляризацию, она одинаково "штрафует" модель за любые ненулевые веса.

$$\large L_1 = \alpha \sum_i |w_i|$$
$$\large L_{1w_i}' = \alpha, \text{  где } w_i\neq 0$$



Посмотрим, как это работает на практике. Для этого применим для этого применим к данным Diabetes линейные модели с регуляризацией,а именнно [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#ridge) (L2 регуляризация), [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#lasso) (L1 регуляризация). Посмотрим на веса признаков при различных значениях $\alpha$.

In [ ]:
from sklearn.linear_model import Ridge, Lasso

n_alphas = 200
alphas = np.logspace(-5, 5, n_alphas)

fig, axs = plt.subplots(figsize=(6, 6), nrows=2, ncols=1)
fig.tight_layout(pad=4.0)

for i, model, model_name, xlim in zip(
    [0, 1], [Ridge, Lasso], ["Ridge (L2)", "Lasso (L1)"], [[0.1, 100_000], [0.0001, 1]]
):
    coefs = []
    coefs_corr = []
    for a in alphas:
        reg = model(alpha=a)
        reg.fit(x_train_norm, y_train_norm)
        coefs_corr.append(reg.coef_[4:6])
        coefs.append(reg.coef_)
    axs[i].plot(alphas, coefs, linestyle="--", alpha=0.5)
    axs[i].plot(alphas, coefs_corr, linewidth=2, label=["s1", "s2"])
    axs[i].set(
        xscale="log", title=model_name, xlabel="alpha", ylabel="weights", xlim=xlim
    )
    axs[i].legend()
    axs[i].grid(True)

Видим, что при маленьком $\alpha$ - веса не нулевые. При очень большом $\alpha$ веса стремятся к 0 для всех признаков - это значит, что вклад в Loss регуляризации становится больше, чем вклад ошибки предсказания.

Нам интересно поведение весов перед признаками  **s1** и  **s2**, т.к. они скоррелированны. В случае L2 веса принимают близкое небольшое значение и сходятся к 0 практически одновременно. В случае L1 - сначала к 0 сходится вес перед **s2**, затем вес перед **s1**.

Интересно отметить, что коэффициенты $\alpha$ при L1 и L2 отличаются по порядку величины. Это связано с тем, что L2 работает с квадратами весов, а L1 с модулями.


Для получения интуиции, что L1-регуляризация позволяет отбирать признаки, часто используют картинку ниже. Функция ошибки задается как сумма ошибки предсказания $L_{\text{err}}$ и регуляризации $L_\text{reg}$.

$$ Loss = L_{\text{err}} + L_\text{reg}$$
где $L_\text{reg}$ - L1 или L2 регуляризация.

Зафиксируем ограничение $L_\text{reg}=1$ и $\alpha = 1$. В пространстве весов это даст для L2 ограничение:
$$w_1^2+w_2^2 = 1$$
Это - единичная окружность (голубая закрашенная область на рисунке ниже слева).

Для L1 ограничение:
$$|w_1|+|w_2| = 1$$
Это - ромб (голубая закрашенная область на рисунке ниже справа).

В реальной задаче значение $L_\text{reg}$ будет меняться при минимизации Loss.


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/loss_landscape_with_regularization.gif" width="800"></center>

<center><em>Source: <a href="https://people.eecs.berkeley.edu/~jrs/189/">Introduction to Machine Learning
</a></em></center>


Вращающиеся овалы показывают, как член Loss, отвечающий за ошибку предсказания $L_{\text{err}}$ (овалы  $L_{\text{err}}=\text{const}$, точка  $\min{L_{\text{err}}}$), изменяется в результате изменения значений признаков и таргета. Оранжевая точка — это минимальное значение для функции Loss с регуляризацией. Видим что:


- Для **L2** оранжевая точка будет кататься **по касательной к окружности**.
- Для **L1** оранжевая точка будет зависать в **уголу ромба**, что соответствует **обнулению веса** одного из признаков.

# Линейная классификация

## Постановка задачи

Теперь попробуем разобраться, что делать в случае классификации.

Рассмотрим задачу классификации на два класса. Например, у нас есть данные о лабораторных мышах. Часть из них имеет нормальную массу тела, а часть — мыши с ожирением.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/svm_mouse_example.png" width="800">

Итак:
1. У нас есть набор данных из $N$ объектов (мышей).
2. Для каждого из объектов (мышей) нам известно признаковое описание объекта в виде набора вещественных чисел (вес, длина от носа до кончика хвоста, возраст и т.д.). То есть объекту под номером $i$ соответствует вектор $\vec x_i$.
3. Также для каждого объекта нам известна истинная метка класса. Мы знаем, что объекту с признаковым описанием $\vec x_i$ соответствует метка класса $y_i$. Будем считать, что метки классов принимают значения:
$$y_i =
\begin{cases}
    1, & \text{для пухляшей}, \\
    0, & \text{для всех остальных}.
\end{cases}$$

Рассмотрим **одномерный пример**. У нас есть данные только по **массе мышей**. Часть из них определена как мыши с нормальной массой тела, а часть — как мыши с ожирением.

Попробуем отделить их друг от друга с использованием линейной регрессии и посмотрим на остатки.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from matplotlib.colors import ListedColormap


custom_cmap = ListedColormap(["#B8E1EC", "#bea6ff", "#FEE7D0"])


def generate_data(total_len=40):
    x = np.hstack(
        [
            np.random.uniform(14, 21, total_len // 2),
            np.random.uniform(24, 33, total_len // 2),
        ]
    )
    y = np.hstack([np.zeros(total_len // 2), np.ones(total_len // 2)])
    return x, y


total_len = 100
np.random.seed(42)

x, y = generate_data(total_len=total_len)
x = x[:, np.newaxis]
regressor = LinearRegression()
regressor.fit(x, y)
y_pred = regressor.predict(x)

x_model = np.linspace(14, 33, 300)
x_model = x_model[:, np.newaxis]
y_model = regressor.predict(x_model)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


def visual_1d_linear_model(x, y, y_pred, x_model, y_model, label):

    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    fig.tight_layout(pad=4.0)

    axs[0].plot(
        x_model,
        y_model,
        label=label,
        linewidth=1,
    )

    axs[0].set(ylabel="y", xlabel="mass, g", title=label)
    sns.scatterplot(x=x[:, 0], y=y, hue=y, s=50, ax=axs[0])
    handles, labels = axs[0].get_legend_handles_labels()
    axs[0].legend(handles, ["Model", "Normal", "Obese"])

    sns.scatterplot(x=x[:, 0], y=y - y_pred, hue=y, s=50, ax=axs[1])
    axs[1].axhline(y=0, color="red")
    axs[1].set(ylabel="dy", xlabel="mass, g", title="Residuals")
    handles, labels = axs[1].get_legend_handles_labels()
    axs[1].legend(handles, ["Normal", "Obese"])
    plt.show()


visual_1d_linear_model(x, y, y_pred, x_model, y_model, "Linear regression")

Остатки сильно скоррелированны. Мы можем внести разделяющее правило
$$y_{pred}>0.5, \text{class} = 1$$
$$y_{pred}\le0.5, \text{class} = 0$$

и считать $Loss=-\text{accurecy}(y_{true}, y_{pred})$, но такое условие не диффернцируемо. Что делать?

## Переход к вероятностям

Давайте попробуем модифицировать простую линейную модель таким образом, чтобы мы могли трактовать выходы модели как вероятность. Начнем с задачи бинарной классификации. Линейная модель задается уравнением:
$$s = f(\vec x) = (\vec{w}, \vec{x}) + b$$

Выходы такой модели принимают значения от $-∞$ до $+∞$.



### Sigmoid



Нам бы хотелось получить на выходе вероятность того, что объект принадлежит к классу $1$. Вероятность принимает значения от $0$ до $1$. Нам нужна функция, которая спроецирует диапазон $(-∞,+∞)$ в диапазон $[0, 1]$.

Такой функцией является сигмоида:
$$p = \sigma(s) = \frac{1}{1+e^{-s}}$$


При $s→-∞$: $p→0$.

При $s→+∞$: $p→1$.

При $s=0$: $p=0.5$.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/scores_to_probability.png" width="500"></center>

Значения $s$ (score) также называют [logit-ом 📚[wiki]](https://en.wikipedia.org/wiki/Logit) (пер. “логарифм”). Это связано с тем, что если выразить logit $s$ через вероятность $p$, то получится формула:
$$s(p) = \log \left(\frac{p}{1-p}\right)$$

### Log loss

Теперь нам нужно решить, какую функцию потерь использовать. **Вероятности** принимают значения **от $0$ до $1$**. Если мы будем использовать **MAE**, [Hinge loss 📚[wiki]](https://en.wikipedia.org/wiki/Hinge_loss) или **MSE**, **максимальным значением ошибки** на указанном диапазоне будет **$1$**. А нам хочется **максимально штрафовать** модель, если она выдает для правильного класса вероятность 0.


Идея **log loss** (ее так же называют Cross-Entropy loss, почему - покажем ниже) в том, что мы хотим притянуть вероятность правильного класса к 1 и бесконечно штрафовать модель если вероятность истинного класса 0.

Для этого можно использовать функцию логорифма. Она гладкая, и для нее: $\log(1) = 0$, $\log(0) = -∞$.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/cross_entropy_plot_loss_with_probability.png" width="800">

Итого мы можем записать функцию ошибок, как:

$$\text{Loss} = - \log(p^y_\text{pred}),$$

где $p^y_\text{pred}$ — предсказанная вероятность для истиного класса. Минус нужен, т.к. $-\log(0) = ∞$.

### Логистическая регрессия

Модель с сигмоидой и log loss называется логистической регрессией. Посмотрим, как логистическая регрессия справляется с задачей классификации.

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x, y)

y_pred = classifier.predict(x)
y_model = classifier.predict_proba(x_model)

visual_1d_linear_model(x, y, y_pred, x_model, y_model[:, 1], "Logistic regression")

## Многоклассовая классификация




Рассмотрим задачу многоглассовой классификации на примере картинок, например, для датасета CIFAR-10 (он встретится вам в задании). У нас есть входное изображение, и мы хотим получить на выходе 10 чисел, обозначающих уверенность модели в принадлежности изображения к конкретному классу. При этом мы знаем, что наши данные устроенны так, что одно изображение может относиться только к одному классу.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/img_to_function_get_scores.png" width="600">







Мы можем считать по logit-у для каждого класса:
$$\begin{matrix} s_1 = (\vec{w_1}, \vec{x}) + b_1\\ s_2 = (\vec{w_2}, \vec{x}) + b_2\\ ... \\ s_n = (\vec{w_n}, \vec{x}) + b_n \end{matrix}$$

Также мы можем записать формулы выше в виде матричного перемножения. Для этого нужно преобразовать матрицу пикселей в вектор признаков (просто вытянув массив в вектор), а строки с весами модели — в матрицу.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/scalar_product_add_bias.png" width="750">

В таком случае для непересекающихся классов результатом классификации будет класс, соответствующий наибольшему logit-у.

В дальнейшем мы будем проходить нейронные сети, которые работают очень похоже.



### SoftMax

[[video] 📺 Объяснение SoftMax от StatQuest](https://www.youtube.com/watch?v=KpKog-L9veg)

Предположим, что наша модель выдала следующие значения logit-ов.

In [ ]:
# fmt: off
logits = [[5.1,    # cat
           3.2,    # car
          -1.7]]   # frog
# fmt: on

Тогда, чтобы узнать, какой класс наша сеть предсказала, мы могли бы просто взять `argmax` от наших `logits`:

In [ ]:
import numpy as np

print("Predicted class = %i (Cat)" % (np.argmax(logits, axis=1).squeeze()))

От argmax нельзя посчитать градиент, так как производная от константы равна 0. Соответственно, если бы мы вставили производную от argmax в градиентный спуск, мы бы получили везде нули, и наша модель бы вообще ничему не научилась.

Кроме того, мы бы хотели получить не logit’ы, а настоящую вероятность $p$ на выходе модели. Да еще и таким образом, чтобы от наших вероятностей можно было посчитать градиент. Для этого мы можем применить к нашим логитам функцию **SoftMax**:
$$\large p(y=k|x=x_i) = \frac{e^{s_k(x_i)}}{\sum_{j=1}^ne^{s_j(x_i)}},$$

где $x_i$ — набор признаков, характеризующий один объект из выборки,
$s_j(x_i)$ — logit для j-го класса для объекта $x_i$.


Функция **SoftMax** вводится на основании двух свойств вероятностей:

1. **Вероятности всегда неотрицательны**. Отобразим наши logit’ы на значения $[0, +∞)$.

> Для этого возведем **экспоненту** (число Эйлера $e=2.71828$) **в степень логита**. В результате мы получим вектор гарантированно неотрицательных чисел (положительное число, возведенное в степень, даже отрицательную, даст положительное значение).

2. Классы не пересекаются, **сумма вероятностей** по всем классам **равна единице**.

>  Мы должны их **нормализовать**, то есть поделить на сумму.

Получаются числа, которые можно интерпретировать, как **вероятности**.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/linear_classifier_softmax.png" width="1000">



$\displaystyle \text{SoftMax}_\text{кошка} = \frac{e^{5.1}}{e^{5.1} + e^{3.2} + e^{-1.7}}$

In [ ]:
def softmax(logits):
    return np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)


print(f"logits: {logits},")
print(f"probabilities: {softmax(logits)}")
print("Sum = %.2f" % np.sum(softmax(logits)))

Можно заметить, что SoftMax никоим образом не поменял порядок значений: самому большому logit'у соответствует самая большая вероятность, а самому маленькому, соответственно, самая маленькая.

#### Практическое вычисление SoftMax

При вычислении экспоненты от выходов модели могут получиться очень большие числа в силу очень высокой скорости роста экспоненты. Этот факт необходимо учитывать, чтобы вычисления SoftMax были численно стабильны:

In [ ]:
from warnings import simplefilter

simplefilter("ignore", category=RuntimeWarning)

f = np.array([[123, 456, 789]])
p = softmax(f)
print(f"logits = {f},\nprobabilities = {p}")

Чтобы регуляризовать вычисление, нам следует предварительно упростить возникающую в вычислении дробь. Для этого мы можем вычесть из каждого $s_i$ положительную константу, чтобы уменьшить значения экспонент. В качестве константы можно выбрать максимальный элемент этого вектора, тогда у нас гарантированно не будет очень больших чисел, и такой способ будет работать более стабильно.

$$\large M = \max_j s_{j}(x_i),$$

$$\large s^\text{new}_{j}(x_i)  = s_{j}(x_i) - M,$$

$$\large \dfrac {e^{s^\text{new}_{k}(x_i)}} {\sum_j e^{s^{new}_{j}(x_i)}}  = \dfrac {e^{s_{k}(x_i) - M }} {\sum_j e^{s_{j}(x_i) - M }} = \dfrac {e^{s_{k}(x_i)}e ^ {-M}} {\sum_j e^{s_{j}(x_i)} e ^ {-M}} = \dfrac {e ^ {-M} e^{s_{k}(x_i)}} {e ^ {-M} \sum_j e^{s_{j}(x_i)} } = \dfrac { e^{s_{k}(x_i)}} { \sum_j e^{s_{j}(x_i)} },$$

где $x_i$ — набор признаков, характеризующий один объект из выборки,
$s_j(x_i)$ — logit для $j$-го класса для объекта $x_i$.


In [ ]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)


f = np.array([[123, 456, 789]])

p = softmax(f)
print(f"logits = {f},\nprobabilities = {p}")

**Замечание:** В реальных вычислениях, SoftMax часто объединяют с Cross-Entropy Loss, это позволяет уменьшить количество операций exp и log.

## Cross-Entropy loss

Итоговая последовательность преобразований будет выглядеть так:

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/linear_model_probability_pipeline.png" width="1000">

Будем использовать log loss. Запишем функцию ошибок, как:

$$\text{Loss} = - \log(p^y_\text{pred}),$$

где $p^y_\text{pred}$ — предсказанная вероятность для истиного класса. Минус нужен, т.к. $-\log(0) = ∞$.

Вернемся к задаче классификации картинок.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/logits_to_scores_to_probabilitys.png" width="900">

На картинке на входе был изображен кот. Мы можем преобразовать метки классов следующим образом:

$$ y → p_\text{true} $$

$$ p_\text{true_i}= \begin{cases} 1 & \text{для i=k, где k - номер истинного класса}, \\0 & \text{для любого } i\neq k.
 \end{cases}$$
Таким образом,
$$ y → p_\text{true} = [[1, 0, 0]]$$


В таком случае функция ошибки выше может быть записанна как [кросс-энтропия 📚[wiki]](https://en.wikipedia.org/wiki/Cross-entropy):
$$\text{Loss} = - \log(p^y_\text{true})= H(p_\text{pred}||p_\text{true})= - \sum^C_{i=1}p_\text{true_i}\cdot \log(p_\text{pred_i}) = -1⋅\log{0.87}-0\cdot\log{0.13}-0\cdot\log{0.001} = - \log(p_\text{cat}) = - \log{0.87} \approx0.14$$

Реализуем это в коде:

In [ ]:
def cross_entropy_loss(pred_prob, true_prob):
    # warning: this code does not take into account the possibility that
    # pred_prob could be equal to zero: add epsilon = 1e-9 to fix it
    return np.sum(-true_prob * np.log(pred_prob)) / pred_prob.shape[0]


# 3 classes 2 items
# fmt: off
logits = np.array([
    [5.1, 3.2, -1.7], # one item
  # [2.1, 6.3,  1.5],  # second item
])
# fmt: on

print(f"Logits = \n{logits}\n")

pred_prob = softmax(logits)
print(f"Predicted Probabilities = \n{pred_prob}\n")

# 3 classes 2 items
# fmt: off
true_prob = np.array([
    [1.0, 0.0, 0.0],  # one item
  # [0.0, 1.0, 0.0],  # second item
])
# fmt: on

print(f"True Probabilities = \n{true_prob}\n")

print(f"Cross-entropy loss = {cross_entropy_loss(pred_prob, true_prob):.3f}")

### Математический смысл

**Кросс-энтропия** имеет глубокий математический смысл в теории информации и статистике. Она связана с [энтропией 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%8D%D0%BD%D1%82%D1%80%D0%BE%D0%BF%D0%B8%D1%8F) $H(P)$ и  [расстоянием Кульбака — Лейблера 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D0%B5_%D0%9A%D1%83%D0%BB%D1%8C%D0%B1%D0%B0%D0%BA%D0%B0_%E2%80%94_%D0%9B%D0%B5%D0%B9%D0%B1%D0%BB%D0%B5%D1%80%D0%B0) $D_{KL}(P||Q)$:


$$H(P||Q) = D_{KL}(P||Q) + H(P)$$


Подробнее об этом можно почитать тут:

[[colab] 🥨 Cross-entropy](https://colab.research.google.com/drive/1DFhT24njhb4LA2g6iJtl2diH-YPtD2Z4
)

### Binary Cross-Entropy

Для задачи **бинарной классификации** или **multi-label классификации** (одному объекту может соответствовать несколько классов) можно использовать модификацию cross-entropy, которую называют **binary cross-entropy**:

$$\text{Loss} =  - \sum^C_{i=1}(p_\text{true_i}\cdot \log(p_\text{pred_i}) + (1-p_\text{true_i})\cdot \log(1-p_\text{pred_i}))$$

В этом случае порог вероятности 0.5 может быть не оптимальным для определения класса. Подробнее про выбор оптимального порога рекомендуем почитать:
* [[blog] ✏️ A Gentle Introduction to Threshold-Moving for Imbalanced Classification](https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/)

### Градиент функции потерь. Кросс-энтропия




Итоговая последовательность преобразований будет выглядеть так:

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/linear_model_probability_pipeline.png" width="1000">

[[blog] ✏️ Cross-Entropy Loss](https://wandb.ai/wandb_fc/russian/reports/---VmlldzoxNDI4NjAw)

Посчитаем производную от функции ошибок. Функция ошибок:

$$ L = - \sum_i p_\text{true_i} \log p_\text{pred_i} = -\sum_i p_\text{true_i} \log(\dfrac {e^{s_i}} {\sum_j e^{s_j}}),$$

где $s_i$ — это логиты классов, получаемые из линейной модели:

$$s_i = w_i x$$

Для расчета градиента будем использовать chain rule:

$$ \dfrac {\partial L} {\partial w_i} = \dfrac {\partial L} {\partial s_i} \dfrac {\partial s_i} {\partial w_i} $$

Градиент логитов по весам:

$$\dfrac {\partial s_i} {\partial w_i} = x$$

У нас только одна истинная метка класса $p_\text{true_k} = 1$, для $i\neq k$ $p_\text{true_k}=0$

$$ L = -1⋅ \log p_\text{pred_k} = - \log(\dfrac {e^{s_k}} {\sum_j e^{s_{j}}})$$



У нас два случая:
1. **Производная по истинному логиту** (значение логита входит в числитель и знаменатель дроби).
2. **Производная по остальным логитам** (значение логита входит только в знаменатель дроби).

Начнем с истинного логита:
1. Производная по $s_{k}$. Вынесем минус, чтобы не потерять:

$$ -L = \log(\dfrac {e^{s_{k}}} {\sum_j e^{s_{j}}}) = \log e^{s_{k}} - \log  \sum_j e^{s_{j}}  = s_{k} - \log  \sum_j e^{s_{j}}$$

$$\dfrac {\partial -L} {\partial s_{k}} = 1 - \dfrac 1 {\sum_j e^{s_{j}}} \cdot \dfrac {\partial {\sum_j e^{s_{j}}}} {\partial s_{k}} = 1 - \dfrac 1 {\sum_j e^{s_{j}}} \cdot \dfrac {\partial e^{s_{k}}} {\partial s_{k}} = 1 - \dfrac {e^{s_{k}}} {\sum_j e^{s_{j}}} = 1 - p_k$$

Вспомним про минус:

$$\dfrac {\partial L} {\partial s_{j}} = p_k - 1 $$

2. Для остальных логитов $i \neq k$. Вынесем минус, чтобы не потерять:

$$ -L = \log(\dfrac {e^{s_{i}}} {\sum_j e^{s_{j}}}) = \log e^{s_{k}} - \log  \sum_j e^{s_{j}}  = s_{k} - \log  \sum_j e^{s_{j}}$$

$$\dfrac {\partial -L} {\partial s_{i}} = - \dfrac 1 {\sum_j e^{s_{j}}} \cdot \dfrac {\partial {\sum_j e^{s_{j}}}} {\partial s_{i}} =  \dfrac 1 {\sum_j e^{s_{i}}} \cdot \dfrac {\partial e^{s_{i}}} {\partial s_{i}} = \dfrac {e^{s_{i}}} {\sum_j e^{s_{j}}} = - p_i$$

Вспомним про минус:

$$\dfrac {\partial L} {\partial s_{j}} = p_i $$

Получаем:
$$ \dfrac {\partial L} {\partial s_{i}}  =  \begin{cases} p_i - 1 & \text{для i=k, где k - номер истинного класса}, \\p_i & \text{для любого } i\neq k.
 \end{cases} $$

Применим **chain rule**:

$$ \dfrac {\partial L} {\partial w_i}  = \dfrac {\partial L} {\partial s_{i}} \dfrac {\partial s_{i}} {\partial w_i}  =  \begin{cases}(p_i - 1)x & \text{для i=k, где k - номер истинного класса}, \\p_ix & \text{для любого } i\neq k.
\end{cases} $$

В коде это будет выглядеть вот так:

In [ ]:
# Input batch of 2 vector with 4 elements
# fmt: off
x = np.array([[1,  2, 3, 4],
              [1, -2, 0, 0]])
# fmt: on

# Weights
W = np.random.randn(3, 4)  # 3 class

# model output
logits = x.dot(W.T)
print("Scores(Logits) \n", logits, "\n")

# Probabilities
probs = softmax(logits)  # defined before
print("Probs \n", probs, "\n")

# Ground true classes
y = [0, 1]

# Derivative
dl_ds = probs.copy()
dl_ds[np.arange(len(y)), y] += -1  # substract one from true class prob
dW = x.T.dot(dl_ds)  # dot product with input

print("Grads dL/dW \n", dW)  # have same shape as W

# Метод опорных векторов (SVM)

Мы рассмотрели вероятностный подход в задаче линейной классификации, он нам еще не раз пригодиться, потому, что в нейронных сетях задача классификации решается именно таким образом.

Помимо вероятностного подхода, существует еще геометрический подход: в нем мы предсказываем не вероятность отношения к классу, а разделяющие классы гиперплоскости. Этот подход имеет свои преимущества. Давайте разбираться.

Замечание: в этом разделе мы не будем углубляться в математику SVM, более подробную информацию вы можете найти в [🥨 [colab] блокноте](https://colab.research.google.com/drive/1jDmQd35fyGauHbcO6jYyMH3-1pUAhSO_).

##1D классификация

Вернемся к задаче классификации мышей. Рассмотрим **одномерный пример**. У нас есть данные только по **массе мышей**. Часть из них определена как мыши с нормальной массой тела, а часть — как мыши с ожирением. Данные выглядят например так:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


custom_cmap = ListedColormap(["#B8E1EC", "#bea6ff", "#FEE7D0"])


def generate_data(total_len=40):
    x = np.hstack(
        [
            np.random.uniform(14, 21, total_len // 2),
            np.random.uniform(24, 33, total_len // 2),
        ]
    )
    y = np.hstack([np.zeros(total_len // 2), np.ones(total_len // 2)])
    return x, y


def plot_data_1d(
    x,
    y,
    total_len=40,
    s=50,
    threshold=None,
    margin=None,
    legend=["Normal", "Obese"],
    marker="o",
):
    ax = sns.scatterplot(x=x, y=np.zeros(len(x)), hue=y, s=s, marker=marker)
    if threshold:
        x_lim, y_lim = ax.get_xlim(), ax.get_ylim()
        XX, YY = np.meshgrid(
            np.linspace(x_lim[0], x_lim[1], 100),
            np.linspace(y_lim[0], y_lim[1], 100),
        )
        pred = np.sign(XX - threshold)
        plt.contourf(XX, YY, pred, alpha=0.3, cmap=custom_cmap)
        ax.axvline(threshold, color="grey")
    if margin:
        for line in margin:
            ax.axvline(line, color="grey", ls="dashed")
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, legend)
    ax.set(xlabel="Mass, g", yticks=[])
    return ax


total_len = 40
np.random.seed(42)
x, y = generate_data(total_len=total_len)
plt.figure(figsize=(5, 3))

ax = plot_data_1d(
    x,
    y,
    total_len=total_len,
)

В случае если классы разделимы (существует пороговое значение, для которого все точки одного множества/класса расположены с одной стороны, а другого — с другой) - задача решается легко: мы находим мышь с ожирением с минимальной массой $\min({m_\text{obese}})$, мышь без ожирения максимальной массой $\max({m_\text{normal}})$. Порог выбираем посередине:

In [ ]:
total_len = 40
np.random.seed(42)
x, y = generate_data(total_len=total_len)
plt.figure(figsize=(5, 3))

normal_limit = x[y == 0].max()  # extreme point for 'normal'
obese_limit = x[y == 1].min()  # extreme point for 'obese'

threshold = np.mean([normal_limit, obese_limit])  # separated with mean value

ax = plot_data_1d(
    x, y, total_len=total_len, threshold=threshold, margin=[normal_limit, obese_limit]
)
plt.plot()

Мы выбрали решение для в котором порог максимально удален от $\min({m_\text{obese}})$ и $\max({m_\text{normal}})$. Такой подход называется Maximum Margin.


In [ ]:
margin_0 = np.abs(normal_limit - threshold)
margin_1 = np.abs(obese_limit - threshold)
print(margin_0, margin_1)

Обратим внимание, что на значение порога влияют только крайние точки в каждом из множеств. Назовем эти точки **опорными**, т.к. в них “упирается” зазор между классами.  

Теперь, пользуясь нашим простым критерием, попробуем классифицировать каких-то новых (тестовых) мышей $\color{orange}{✭}$ $\color{blue}{✭}$:

In [ ]:
x_test = np.random.uniform(14, 30, 5)


def classify(x, threshold=21.5):
    y = np.zeros_like(x)
    y[x > threshold] = 1
    return y


plt.figure(figsize=(5, 3))
ax = plot_data_1d(
    x, y, total_len=total_len, threshold=threshold, margin=[normal_limit, obese_limit]
)
ax = plot_data_1d(
    x_test,
    classify(x_test, threshold=threshold),
    total_len=total_len,
    s=500,
    marker="*",
)

Получилось неплохо.

## Многомерная классификация

Мы умеем решать задачу классификации в одномерном случае. В многомерном случае попробуем свести задачу к одномерной. Для этого мы будем подбирать вектор весов и смещение чтобы точки преобразовывались, как на картинке:

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/svm_hard_margin.png" width="1000">

Подобрать такое преобразование в многомерной задаче — значит провести **разделяющую гиперплоскость** так, чтобы:
1. Объекты разных классов лежали **по разные** стороны от этой плоскости.
2. **Ближайшие** к плоскости **объекты** были от нее как можно **дальше**.



**Гиперплоскость однозначно задается вектором нормали $\vec w$ и смещением $b$.** Скалярное произведение на вектор весов $\vec w$ - это проекция на вектор нормали разделяющей плоскости. Побирая $\vec w$ и $b$ мы сводим задачу у одномерной.

Зазор между классами будет опираться в объекты, которые мы назовем **опорными векторами**. Именно эти объекты проектируются преобразованием в точки 1 и -1, это обеспечивает удобство при математической записи преобразования и поиске решения (мера проекции опорного вектора = 1). Для более глубокого понимания SVM рекомендуем видео:
* [[video] 📺 MIT: Support Vector Machines](https://www.youtube.com/watch?v=_PwhiWxHK8o)

Математика Support Vector Machines (в которую мы не будем погружаться в рамках этой лекции) крутится вокруг этих объектов. Функцию потерь и решающее правило [можно расписать ✏️[blog]](https://www.geeksforgeeks.org/dual-support-vector-machine/) через скалярное произведение с опорными векторами. Грубо мы можем сказать, что скалярное произведение - это ["мера подобия" 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C#:~:text=%D0%9A%D0%BE%D1%81%D0%B8%D0%BD%D1%83%D1%81%D0%BD%D0%BE%D0%B5%20%D1%81%D1%85%D0%BE%D0%B4%D1%81%D1%82%D0%B2%D0%BE%20%E2%80%94%20%D1%8D%D1%82%D0%BE%20%D0%BC%D0%B5%D1%80%D0%B0%20%D1%81%D1%85%D0%BE%D0%B4%D1%81%D1%82%D0%B2%D0%B0,%D0%B8%D0%B7%D0%BC%D0%B5%D1%80%D0%B5%D0%BD%D0%B8%D1%8F%20%D0%BA%D0%BE%D1%81%D0%B8%D0%BD%D1%83%D1%81%D0%B0%20%D1%83%D0%B3%D0%BB%D0%B0%20%D0%BC%D0%B5%D0%B6%D0%B4%D1%83%20%D0%BD%D0%B8%D0%BC%D0%B8.) между векторами и "сравнивая" классифицируемый объект с опорными векторами мы можем определить его класс.

Рассмотрим задачу классификации мышей, в которой мы измерили не только вес мышей, но и их длину от хвоста до носа.

In [ ]:
from sklearn.datasets import make_blobs


def generate_2d_data(total_len=40):
    x, y = make_blobs(n_samples=total_len, centers=2, random_state=42)
    x[:, 0] += 10
    x[:, 1] += 20
    return x, y


total_len = 40
x, y = generate_2d_data(total_len=total_len)

fig = plt.figure(figsize=(5, 3))
ax = sns.scatterplot(x=x[:, 0], y=x[:, 1], hue=y, s=50)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ["Normal", "Obese"])
ax.set(xlabel="Mass, g", ylabel="Length, cm")
plt.show()

Применим к мышкам метод `svm` из библиотеки Sklearn:

In [ ]:
from sklearn import svm

# Code for illustration, later we will understand how it works
# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel="linear", C=1000)
clf.fit(x, y)

fig, axs = plt.subplots(1, 2, figsize=(10, 3))

# first fig
sns.scatterplot(x=x[:, 0], y=x[:, 1], hue=y, s=50, ax=axs[0])
handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles, ["Normal", "Obese"])
axs[0].set(xlabel="Mass, g", ylabel="Length, cm")

# plot the decision function
delta = 0.5
# create grid to evaluate model
YY, XX = np.meshgrid(
    np.linspace(x[:, 1].min() - delta, x[:, 1].max() + delta, 30),
    np.linspace(x[:, 0].min() - delta, x[:, 0].max() + delta, 30),
)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)
pred = np.sign(Z)
axs[0].contourf(XX, YY, pred, alpha=0.3, cmap=custom_cmap)

# plot decision boget_xlimundary and margins
axs[0].contour(
    XX,
    YY,
    Z,
    colors="k",
    levels=[-1, 0, 1],
    alpha=0.5,
    linestyles=["--", "-", "--"],
)
# plot support vectors
axs[0].scatter(
    clf.support_vectors_[:, 0],
    clf.support_vectors_[:, 1],
    s=100,
    linewidth=1,
    facecolors="none",
    edgecolors="k",
)

# second fig
dec_val = clf.decision_function(x)
sns.scatterplot(x=dec_val, y=np.zeros(len(x)), hue=y, ax=axs[1])

x_lim, y_lim = axs[1].get_xlim(), axs[1].get_ylim()
XX, YY = np.meshgrid(
    np.linspace(x_lim[0], x_lim[1], 100), np.linspace(y_lim[0], y_lim[1], 100)
)
pred = np.sign(XX)
axs[1].contourf(XX, YY, pred, alpha=0.3, cmap=custom_cmap)

axs[1].axvline(0, color="grey")
axs[1].axvline(-1, color="grey", ls="dashed")
axs[1].axvline(1, color="grey", ls="dashed")
handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles, ["Normal", "Obese"])
axs[1].set(xlabel="wx+b", yticks=[])

sv = clf.decision_function(clf.support_vectors_)
axs[1].scatter(
    sv, np.zeros_like(sv), s=100, linewidth=1, facecolors="none", edgecolors="k"
)
plt.show()

Круги - опопрные вектора. Пунктиром показан зазор.

**Практические советы по использованию SVM:**

* SVM делает **геометрическое разделение данных**, поэтому для адекватной работы модели важна **нормализация**.
* В случае **дисбаланса классов** полезно использовать параметры `class_weight` и `sample_weight` ([подробнее 🛠️[doc]](https://scikit-learn.org/stable/modules/svm.html#unbalanced-problems)).
* SVM работает и для классификации неразделимых данных (вводится ограничение, на сколько классы могут смешиваться, обычно за это отвечает параметр `C`).
* SVM может давать хорошее решение при небольшом количестве данных, в этом случае стоит попробовать **различные ядра** (про ядра вы узнаете ниже).

# Обобщенные линейные модели

## Полиномиальная модель

Данные не всегда могут быть **хорошо разделены (гипер)плоскостью**. Например: у нас есть данные по дозировке лекарства и 2 класса — пациенты, которые поправились, и те, которым лучше не стало.

In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt


def generate_patients_data(total_len=40):
    x = np.random.uniform(0, 50, total_len)
    y = np.zeros_like(x)
    y[(x > 15) & (x < 35)] = 1
    return x, y


def plot_data(x, y, total_len=40, s=50):
    plt.figure(figsize=(5, 3))
    ax = sns.scatterplot(x=x, y=np.zeros(len(x)), hue=y, s=s)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ["Sick", "Recover"])
    ax.set(xlabel="dose, mg")
    return ax


total_len = 40
x, y = generate_patients_data(total_len=total_len)
ax = plot_data(x, y, total_len=total_len)
plt.show()

Соответственно, мы не можем найти такое пороговое значение, которое будет разделять наши классы на больных и здоровых, а, следовательно, и Support Vector Classifier работать тоже не будет.  Для начала давайте преобразуем наши данные таким образом, чтобы они стали 2-хмерными. В качестве значений по оси Y будем использовать дозу, возведенную в квадрат (**доза**$^2$).

In [ ]:
def plot_data(x, y, total_len=40, s=50):
    plt.figure(figsize=(5, 3))
    ax = sns.scatterplot(x=x[0, :], y=x[1, :], hue=y, s=s)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ["Sick", "Recover"])
    ax.set(xlabel="Dose, mg")
    ax.set(ylabel="Dose$^2$")
    return ax


total_len = 40
x_1, y = generate_patients_data(total_len=total_len)
x_2 = x_1**2
x = np.vstack([x_1, x_2])

plot_data(x, y, total_len=40, s=50)
plt.show()

Теперь мы можем вновь использовать Support Vector Classifier для классификации.

In [ ]:
plot_data(x, y, total_len=40, s=50)

x_arr = np.linspace(0, 50, 50)
xs = [x[0, :][y == 1].min(), x[0, :][y == 1].max()]
ys = [x[1, :][y == 1].min(), x[1, :][y == 1].max()]

# Calculate the coefficients.
coefficients = np.polyfit(xs, ys, 1)

# Let's compute the values of the line...
polynomial = np.poly1d(coefficients)
y_axis = polynomial(x_arr)

# ...and plot the points and the line
plt.plot(x_arr, y_axis, "r--")
plt.show()

## Kernel SVM

Основная идея **Kernel SVM** состоит в том, что **мы можем перейти в пространство большей размерности, в котором данные будут линейно разделимы**.

 ### Обоснование Kernel SVM

<img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/svm_kernel_function.png" width="700">


Мы уже говорили, что **решающее правило SVM** можно записать через **скалярное произведение с опорными векторами**. В  Kernel SVM скалярное произведение заменяется на другую функцию подобия, которая называется **ядром/kernel**.

[Подробнее 📚[book]](https://people.csail.mit.edu/dsontag/courses/ml13/slides/lecture6.pdf).

### Примеры ядер

Для демонстрации возможностей Kernel SVM создадим датасет, который не разделяется линейными моделями. Для этого воспользуемся функцией `sklearn.datasets.make_circles`.

In [ ]:
from sklearn.datasets import make_circles

x, y = make_circles(n_samples=500, factor=0.3, noise=0.05, random_state=42)

Полученный датасет представляет собой две окружности с разными радиусами и общим центром, относящиеся к разным классам.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

sns.scatterplot(
    x=x[:, 0],
    y=x[:, 1],
    hue=y,
    s=50,
    ax=ax,
    palette=sns.color_palette(["#2DA9E1", "#F9B041"]),
)
plt.show()

Напишем функцию визуализации разделяющего правила для SVM модели:

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay


def plot_svm(x, y, clf):
    dull_cmap = ListedColormap(["#B8E1EC", "#FEE7D0"])
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))

    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        x,
        response_method="predict",
        cmap=dull_cmap,
        alpha=0.8,
        xlabel="feature 1",
        ylabel="feature 2",
        ax=ax,
    )

    sns.scatterplot(
        x=x[:, 0],
        y=x[:, 1],
        hue=y,
        s=50,
        ax=ax,
        palette=sns.color_palette(["#2DA9E1", "#F9B041"]),
    )
    plt.show()

Первое ядро, которое мы рассмотрим, линейное. Оно задается формулой:
$$\large K(\vec x_i, \vec x_j) = (\vec x_i, \vec x_j)$$

Линейным ядром является скалярное произведение векторов.

Линейное ядро не способно справиться с такой задачей:

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel="linear")
clf.fit(x, y)
plot_svm(x, y, clf)

Следующее ядро, реализованное в библиотеке Sklearn — полиномиальное, оно задается формулой:
$$K(\vec x_i, \vec x_j) = (\gamma (\vec x_i, \vec x_j)+r)^d,$$
где $d$ — настраиваемый параметр: степень полинома `degree`.

Попробуем применить полиномиальное ядро к нашим данным:

In [ ]:
clf = svm.SVC(kernel="poly")
clf.fit(x, y)
plot_svm(x, y, clf)

Полиномиальное ядро действует не совсем как полиномиальная модель.
У модели не получилось разделить данные. Это связано с тем, что значение `degree` по умолчанию равно 3, поставим степень полинома 2:

In [ ]:
clf = svm.SVC(kernel="poly", degree=2)
clf.fit(x, y)
plot_svm(x, y, clf)

Вывод, который тут стоит сделать: для получения **оптимального результата** бывает полезным **настроить параметры ядра** с учетом даных.

Самым  популярным ядром SVM является ядро радиальных базисных функций RBF, или гауссово ядро. Оно получено из гауссова  распределения, а гауссово распределение характерно для большого количества измеряемых величин. Данное ядро задается формулой:

$$\large K(\vec x_i, \vec x_j) = e^{-\gamma{||\vec x_i - \vec x_j||^2}}$$

Настраиваемыми параметрами модели являются `C` и `gamma`. `C` определяет степень гладкости поверхности принятия решений: чем больше `C`, тем сложнее поверхность и **выше вероятность переобучения** (про переобучение поговорим ниже), `gamma` определяет степень влияния одного обучающего примера на разделяющее правило ([подробнее 🛠️[doc]](https://scikit-learn.org/stable/modules/svm.html#parameters-of-the-rbf-kernel)).

SVM может проверять пространства признаков бесконечного размера, если для такого пространства существует kernel function. RBF ядро как раз [соответствует 📚[book]](https://pages.cs.wisc.edu/~matthewb/pages/notes/pdf/svms/RBFKernel.pdf) такому случаю бесконечномерного пространства признаков.

In [ ]:
clf = svm.SVC(kernel="rbf")
clf.fit(x, y)
plot_svm(x, y, clf)

Также в Sklearn реализовано `sigma` ядро. Оно интересно больше с исторической точки зрения, т.к. эквивалентно модели нейрона — Перцептрону, о котором вы узнаете на 5-й лекции. [На практике 🎓[article]](https://home.work.caltech.edu/~htlin/publication/doc/tanh.pdf) оно в большинстве случаев проигрывает RBF ядру.

$$\large K(\vec x_i, \vec x_j) = \tanh (\gamma(\vec x_i, \vec x_j)+r)$$

#Наивный Байесовский классификатор

Можно построить модель классификатора, которая будет напрямую оценивать вероятность принадлежности объекта к интересующему нас классу не на основе скалярных произведений и логитов, а на основе априорной информации о распределении объектов по классам в обучающей выборке. Это легко продемонстрировать на следующих данных.

## Пример на табличных данных

Загрузим `DataFrame` датасета [Wine 🛠️[doc]](https://archive.ics.uci.edu/ml/datasets/wine), который являлся примером табличных данных в первой лекции:

In [ ]:
import sklearn
from sklearn.datasets import load_wine

# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine

# Download dataset
features, class_labels = load_wine(
    return_X_y=True, as_frame=True
)  # also we can get data in Bunch (dictionary) or pandas DataFrame

wine_dataset = features
wine_dataset["target"] = class_labels

wine_dataset.head()

Датасет содержит объекты 3 различных классов:

In [ ]:
wine_dataset.target.unique()

Возьмём первый признак `alcohol`. По имеющийся таблице с данными легко построить функцию распределения $f(x)$, которая будет задавать вероятность $p(\text{alcohol} = x)$, и найти среди наших данных бутылку вина с параметром `alcohol`, равным $x$ (слева на графике ниже).

Т.к. у нас три класса, мы можем построить распределение объектов в обучающей выборке по признаку `alcohol` отдельно для каждого из этих трёх классов. Эти распределения зададут нам условную вероятность $p(\text{alcohol} = x |\text{target} = i)$ того, что объект имеет значение признака `alcohol`, равное $x$, при условии, что он относится к одному из классов с номером $i$ (справа на графике ниже).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.kdeplot(wine_dataset, x="alcohol", fill=True, ax=axes[0])
axes[0].set_title("p(alcohol=x)")

sns.kdeplot(
    wine_dataset,
    x="alcohol",
    hue="target",
    palette=sns.color_palette(["#5D5DA6", "#2DA9E1", "#F9B041"]),
    fill=True,
    ax=axes[1],
)
axes[1].set_title("p(alcohol=x|target=i)")
plt.show()

Посмотрев на плотности распределений по классам (график справа), мы можем предположить, что бутылка с значением $\text{alcohol} = 11.3$ будет относиться к 1 классу.

На языке формул наш “метод пристального вглядывания” можно записать с помощью формулы для условной вероятности по [теореме Байеса 📚[wiki]](https://en.wikipedia.org/wiki/Bayes'_theorem):

$$\large p(\text{target} = i | \text{alcohol} = x) = \frac{p(\text{alcohol} = x | \text{target} = i )p(\text{target} = i )}{p(\text{alcohol} = x)},$$

где $p(\text{target} = i)$ — доля объектов класса $i$ в датасете, а $p(\text{target} = i | \text{alcohol} = x)$ — вероятность того, что объект принадлежит классу $i$, при условии того, что признак `alcohol` у него принимает значение $x$.


Мы поняли, откуда в названии метода **Байес**. Теперь разберемся, **почему он “наивный”**.

Мы использовали только один признак: `alcohol`. Всего же у нас 13 признаков.

$$\large p(\text{target} = i |\text{features} = \vec x ) = \frac{p(\text{features} = \vec x | \text{target} = i )p(\text{target} = i )}{p(\text{features} = \vec x)}$$

**“Наивность”** Байеса состоит в том, что эта модель будет рассматривать признаки как **независимые случайные величины**:

$$\large p(\text{features} = \vec x)=p(\text{feature}_1 = x_1) \cdot p(\text{feature}_2 = x_2)...p(\text{feature}_n = x_n)$$

Если мы решаем задачу классификации на $k$ классов, то для объекта с набором признаком $\vec x$ по формуле выше мы получим $k$ чисел, характеризующих вероятность принадлежности данного объекта к различным классам. Для финального принятия решения нам останется выбрать тот класс, для которого вероятность принадлежности наивысшая:

$$\large \text{prediction} = \underset{i}{\text{argmax}}{\left(p(\text{target} = i |\text{features} = \vec x )\right)}.$$

Вернёмся к нашему датасету Wine и попробуем решить задачу классификации для него при помощи предложенного алгоритма.

Как обычно, разделим наш датасет на тренировочную и валидационную выборки:

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test data
x_train, x_test, y_train, y_test = train_test_split(
    features.values, class_labels.values, test_size=0.25, random_state=42
)

Возьмём реализацию Наивного Байесовского классификатора `GaussianNB` из [библиотеки Sklearn 🛠️[doc]](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB). `GaussianNB` **использует оценку распределения признаков с помощью** [Гауссового распределения 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5). Обучим её на тренировочном датасете и измерим качество на отложенной валидационной выборке:

In [ ]:
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB

# Train the model
model = GaussianNB()
model.fit(x_train, y_train)

# Calculate F1_score
pred = model.predict(x_test)
f1_score(y_test, pred, average="macro")

Благодаря простоте модельного датасета Wine наша наивная статистическая модель показала отличное качество работы. Это связано с “простотой” датасета — признаки классов имеют унимодальные распределения (один пик на плотности распределения), для более сложных данных (многомодальные распределения) такого не будет.

Тем не менее, подход к решению задачи классификации, связанный с построением модели предсказания принадлежности объекта к имеющимся классам, оказался конструктивным.

Идея применения наивного Байесовского классификатора в NLP задаче:
* [[colab] 🥨 Naive Bayes NLP](https://colab.research.google.com/drive/1JUMs9TBWxlCTsqw2T-y3oxMe2WZxLYtJ)

# Практические особенности работы с линейными моделями

## Нормализация данных

Мы уже обсудили, зачем нужна нормализация данных для линейной модели. В данном разделе мы обсудим виды нормализации более подробно.

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков соответствует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30-ти признакам обучить модель определять тип ткани (злокачественная или доброкачественная).


In [ ]:
import sklearn.datasets

cancer = sklearn.datasets.load_breast_cancer()  # load data

x = cancer.data  # features
y = cancer.target  # labels(classes)
print(f"x shape: {x.shape}, y shape: {y.shape}")
print(f"x[0]: \n {x[0]}")
print(f"y[0]: \n {y[0]}")

In [ ]:
import pandas as pd

cancer_df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
cancer_df

Теперь давайте посмотрим на сами данные. У нас есть 569 строк, в каждой из которой по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд).

In [ ]:
cancer_df.describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные диапазоны  значений.

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt


plt.figure(figsize=(6, 5))
ax = sns.boxenplot(
    data=cancer_df,
    orient="h",
    palette="Set2",
    linewidth=0.4,
    flier_kws={"marker": "o", "s": 3},
    line_kws={"linewidth": 1},
)
ax.set(xscale="log", xlim=(1e-4, 1e4), xlabel="Values", ylabel="Features")
plt.show()

Линейная модель представляет собой **сумму взвешенных признаков**. Если мы приведем признаки к **единому масштабу**, мы сможем **оценить их вклад в модель** по значениям весов. Кроме того, работать с признаками в одном диапазоне вычислительно удобно. Для этого будем использовать нормализацию.


### Выбор Scaler

**Нормализацией** называется процедура приведения входных данных к **общим значениям математических статистик**.

Нормализация строит **взаимно однозначное соответствие** между некоторыми размерными величинами (которые измеряются в метрах, килограммах, годах и т. п.) и их безразмерными аналогами. Исходные значения **можно восстановить**, зная статистики оригинальных данных и правило, по которому делалась нормализация.  


Часто используют следующие варианты нормализации:  **`MinMaxScaler`**, **`StandardScaler`**, **`RobustScaler`**.

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные из имеющегося диапазона значений в диапазон от $0$ до $1$. Может быть полезно, если нужно выполнить преобразование, в котором отрицательные значения не допускаются (например, масштабирование RGB пикселей).

$$\large z_i=\frac{X_i-X_{\min}}{X_{\max}-X_{\min}},$$

где $z_i$ — масштабированное значение, $X_i$ — текущее значение, $X_{\min}$ и $X_{\max}$ — минимальное и максимальное значения имеющихся данных.

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение $0$ и стандартное отклонение $1$. Большинство значений будет находиться в диапазоне от $-1$ до $1$. Это стандартная трансформация, и она применима во многих ситуациях.

$$\large z_i=\frac{X_i-u}{s},$$

где $u$ — среднее значение (или $0$ при `with_mean=False`), $s$ — стандартное отклонение (или $0$ при `with_std=False`).

И `StandardScaler`, и `MinMaxScaler` чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях*. $k$-й процентиль — это величина, равная или не превосходящая $k$ процентов чисел во всём имеющемся распределении. Например, 50-й процентиль (медиана) распределения таков, что 50% чисел из распределения не меньше данного числа.

Соответственно, `RobustScaler` не зависит от небольшого числа очень больших предельных выбросов (outliers). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков.

$$\large z_i=\frac{X_i-X_\text{median}}{IQR},$$

где $X_\text{median}$ — значение медианы, $IQR$ — межквартильный диапазон, равный разнице между 75-ым и 25-ым процентилями.

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для случайного набора признаков.

In [ ]:
import random

random.seed(0)
random_names = random.sample(list(cancer.feature_names), 8)
cut_df = cancer_df[random_names]

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler


def plot_norm(df, ax, title):
    sns.boxenplot(
        df,
        orient="h",
        palette="Set2",
        ax=ax,
        linewidth=0.2,
        flier_kws={"marker": "o", "s": 5},
        line_kws={"linewidth": 1},
    )
    ax.set(xlabel="Values", title=title)


fig, axs = plt.subplots(2, 2, figsize=(10, 7))

plot_norm(cut_df, axs[0][0], "Original")
axs[0][0].set(xscale="log", xlim=(1e-4, 1e4))
min_max_x = MinMaxScaler().fit_transform(cut_df)
plot_norm(pd.DataFrame(min_max_x, columns=random_names), axs[0][1], "MinMax")

std_x = StandardScaler().fit_transform(cut_df)
plot_norm(pd.DataFrame(std_x, columns=random_names), axs[1][0], "Standard")

rob_x = RobustScaler().fit_transform(cut_df)
plot_norm(pd.DataFrame(rob_x, columns=random_names), axs[1][1], "Robust")

plt.subplots_adjust(wspace=0.55, hspace=0.35)

plt.show()

Перед выбором нормализации **важно разобраться с природой выбросов**. Для этого нужно посмотреть на выбросы с точки зрения эксперта и попробовать определить, являются ли выбросы ошибкой при сборе данных или редкими случаями, которые необходимо сохранить.

Мы не являемся экспертами в медицине и мало знаем о данных, поэтому будем считать, что наши признаки имеют распределение, близкое к нормальному. Поэтому мы будем использовать `StandardScaler`. `StandardScaler` часто используют как нормировку по умолчанию.

In [ ]:
x_norm = StandardScaler().fit_transform(cancer_df)  # scaled data

Видим, что они стали намного более сравнимы между собой.

In [ ]:
pd.DataFrame(x_norm, columns=cancer.feature_names).describe()

In [ ]:
plt.figure(figsize=(6, 5))
ax = sns.boxenplot(
    data=pd.DataFrame(x_norm, columns=cancer.feature_names),
    orient="h",
    palette="Set2",
    linewidth=0.4,
    flier_kws={"marker": "o", "s": 3},
    line_kws={"linewidth": 1},
)
ax.set(xlabel="Values")
plt.show()

## Борьба с переобучением

__Сложность модели__ (*model complexity*) — важный гиперпараметр. В частности, для линейных моделей сложность может быть представлена **количеством параметров**, например, для полиномиальных моделей — степенью полинома.

Сложность модели связана с **ошибкой обобщения** (_generalization error_):
- **Cлишком простой модели** не будет хватать **количества параметров для обобщения сложной закономерности в данных**, что приведёт к большой ошибке обобщения.
- **Избыточная сложность модели** также приводит к большой ошибке обобщения за счет того, что в силу своей сложности модель начинает **пытаться искать закономерности в шуме**, добиваясь большей точности на тренировочных данных, теряя при этом часть обобщающей способности.



<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/model_complexity.png" width="500">

Параметры модели задают некоторую **аппроксимацию целевой функции**. Аппроксимировать целевую функцию можно несколькими способами, например:
1. Использовать все имеющиеся данные и провести ее строго **через все точки**, которые нам известны ($f1$ на картинке);
2. Использовать более простую функцию (в данном случае, линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым **общим закономерностям**, которые у них есть ($f2$ на картинке).

Характерной чертой переобучения является первый сценарий, и сопровождается он, как правило, **большими весами**. Поэтому регуляризацию используют для борьбы с переобучением.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/l2_regularization.png" width="300">

Проиллюстрируем описанное явление на примере полиномиальной модели:

In [ ]:
import numpy as np

x = np.linspace(0, 2 * np.pi, 10)
y = np.sin(x) + np.random.normal(scale=0.25, size=len(x))
x_true = np.linspace(0, 2 * np.pi, 200)
y_true = np.sin(x_true)

plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=50, facecolors="none", edgecolors="b", label="noisy data")
plt.plot(x_true, y_true, c="lime", label="ground truth")
plt.legend()
plt.show()

Попробуем аппроксимировать имеющуюся зависимость с помощью полиномиальной модели, используя шумные данные в качестве тренировочных данных:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

x_train = x.reshape(-1, 1)

fig = plt.figure(figsize=(10, 5))

for i, degree in enumerate([0, 1, 3, 9]):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    model.fit(x_train, y)
    y_plot = model.predict(x_true.reshape(-1, 1))

    fig.add_subplot(2, 2, i + 1)
    plt.plot(x_true, y_plot, c="red", label=f"M={degree}")
    plt.scatter(x, y, s=50, facecolors="none", edgecolors="b")
    plt.plot(x_true, y_true, c="lime")
    plt.legend()
plt.show()

Модель **переобучается, подстраиваясь под тренировочную выборку**. В полиноме степень/количество весов — это гиперпараметр, который можно подбирать на кросс-валидации, но **ограничивая количество параметров**, мы накладываем **ограничение на обобщающую способность модели** в целом. Вместо этого можно оставить модель сложной, но использовать **регуляризатор**, который будет заставлять модель отдавать предпочтение более простому обобщению.

In [ ]:
from sklearn.linear_model import Ridge


model = make_pipeline(PolynomialFeatures(9), LinearRegression())
model_ridge = make_pipeline(PolynomialFeatures(9), Ridge(alpha=0.1))

model.fit(x_train, y)
y_plot = model.predict(x_true.reshape(-1, 1))

model_ridge.fit(x_train, y)
y_plot_ridge = model_ridge.predict(x_true.reshape(-1, 1))

plt.figure(figsize=(5, 3))
plt.plot(x_true, y_plot, c="red", label=f"M={degree}")
plt.plot(x_true, y_plot_ridge, c="black", label=f"M={degree}, alpha=0.1")
plt.scatter(x, y, s=50, facecolors="none", edgecolors="b")
plt.plot(x_true, y_true, c="lime", label="ground truth")
plt.legend()
plt.show()

poly_coef = model[1].coef_

eq = f"y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x"
for i in range(2, 10):
    eq += f"+{round(poly_coef[i], 2)}*x^{i}"

print("Without regularization: ", eq)

poly_coef = model_ridge[1].coef_

eq = f"y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x"
for i in range(2, 10):
    eq += f"+{round(poly_coef[i], 2)}*x^{i}"

print("With regularization: ", eq)

Видно, что одним из "симптомов" переобучения являются аномально большие веса. Модель Ridge Regression, показанная в примере выше, использует L2-регуляризацию для борьбы с этим явлением.


<font size="6">Литература</font>

<font size="5">Линейная регрессия:</font>

* [[book] 📚 Метрики классификации и регрессии](https://academy.yandex.ru/handbook/ml/article/metriki-klassifikacii-i-regressii)

<font size="5">Метод градиентного спуска:</font>
* [[book] 📚 Градиентный спуск](https://www.tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter1_gradient-descent.html)
* [[book] 📚 Стохастический градиентный спуск](https://tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter2_stochastic-gradient-descent.html)

<font size="5">SVM:</font>
* [[video] 📺 Хорошее объяснение SVM](https://www.youtube.com/watch?v=_PwhiWxHK8o)
* [[book] 📚 SVM на стэнфордском курсе ](https://cs231n.github.io/linear-classify/#svm)

<font size="5">Обобщенные линейные модели:</font>
* [[blog] ✏️ Дуальная форма SVM](https://www.geeksforgeeks.org/dual-support-vector-machine/)
* [[git] 🐾 Продвинутый алгоритм, основанный на SVM](https://github.com/IvanoLauriola/MKLpy)

<font size="5">Вероятностный подход в задаче классификации:</font>
* [[video] 📺 Naive Bayes in NLP](https://www.youtube.com/watch?v=O2L2Uv9pdDA)
* Latent Dirichlet Allocation:
 * [[video] 📺 Latent Dirichlet Allocation (Part 1 of 2)](https://www.youtube.com/watch?v=T05t-SqKArY),
 * [[video] 📺 Training Latent Dirichlet Allocation: Gibbs Sampling (Part 2 of 2)](https://www.youtube.com/watch?v=BaM1uiCpj_E).
* [[video] 📺 Объяснение SoftMax от StatQuest](https://www.youtube.com/watch?v=KpKog-L9veg)
* [[video] 📺 Объяснение KL Divergence](https://www.youtube.com/watch?v=SxGYPqCgJWM)